In [1]:
import json
import os
import numpy as np
from scipy import stats as stats_mode
import shutil
import cv2
import random
import pandas as pd
import re
lst_of_folders = ['without_box']

In [2]:
random.seed(10)

In [3]:
def add_num(n):
    c=0
    i=n
    while(n>0):
        n=n//10
        c+=1
    if c==1 or c==3:
        if i%10 == 1:
            return "st"
        elif i%10 == 2:
            return "nd"
        elif i%10 == 3:
            return "rd"
        elif i%10 == 4:
            return "th"
        elif i%10 == 5:
            return "th"
        elif i%10 == 6:
            return "th"
        elif i%10 == 7:
            return "th"
        elif i%10 == 8:
            return "th"
        elif i%10 == 9:
            return "th"
    elif c==2:
        v=i%10
        if v==0:
            return "th"
        if v!=0:
            if i%10 == 1:
                return "st"
            elif i%10 == 2:
                return "nd"
            elif i%10 == 3:
                return "rd"
            elif i%10 == 4:
                return "th"
            elif i%10 == 5:
                return "th"
            elif i%10 == 6:
                return "th"
            elif i%10 == 7:
                return "th"
            elif i%10 == 8:
                return "th"
            elif i%10 == 9:
                return "th"

### QA generation

In [12]:
for i in lst_of_folders:
    copy_folder = "C:/Users/pande/Desktop/test_2023/copy_JSON/"+i+"/"
    qa_folder = "C:/Users/pande/Desktop/test_2023/QA/"+i+"_qa/"
    if os.path.exists(qa_folder):
        shutil.rmtree(qa_folder)
    if not os.path.exists(qa_folder):
        os.mkdir(qa_folder)
    for file in os.listdir(copy_folder):
        location = copy_folder+file
        qa=list()
        with open(location,'r') as f:
            js = json.load(f)

            if 'task1' in js.keys():
                if js['task1']!=None:
                    title_horizontal = "yes" if "horizontal" in js['task1']['output']['chart_type'].split(" ") else "no"

            if 'task7' in js.keys():
                if 'are_there_zero_values' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        q=dict()
                        q['taxonomy id'] = '3a'
                        q['QID'] = 1
                        q['question'] = "Does the chart contain any zero data values?"
                        q['answer'] = js['task7']['are_there_zero_values']
                        qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            q=dict()
                            q['taxonomy id'] = '3a'
                            q['QID'] = 1
                            q['question'] = "Does the chart contain any zero data values?"
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(round(j[1],2))
                            flag = 1
                            for i in dat:
                                if i==0.00:
                                    flag=0
                            if js['task7']['are_there_zero_values']=='yes' or flag==0:
                                q['answer'] = 'yes'
                            else:
                                q['answer'] = 'no'
                            qa.append(q)
                    else:
                        q=dict()
                        q['taxonomy id'] = '3a'
                        q['QID'] = 1
                        q['question'] = "Does the chart contain any zero data values?"
                        q['answer'] = js['task7']['are_there_zero_values']
                        qa.append(q)

            if 'task7' in js.keys():
                if 'legend_position' in js['task7'].keys() :
                    q=dict()
                    q['taxonomy id'] = '2e'
                    q['QID'] = 3
                    q['question'] = "Where does the legend appear in the chart?"
                    q['answer'] = js['task7']['legend_position']
                    qa.append(q)

            if 'task7' in js.keys():
                if 'legend_length' in js['task7'].keys() :
                    q=dict()
                    q['taxonomy id'] = '2g'
                    q['QID'] = 4
                    q['question'] = "How many legend labels are there?"
                    q['answer'] = js['task7']['legend_length']
                    qa.append(q)

            if 'task7' in js.keys():
                if 'legend_labels_orientation' in js['task7'].keys() :
                    q=dict()
                    q['taxonomy id'] = '2g'
                    q['QID'] = 5
                    q['question'] = "What is the orientation of legend labels?"
                    q['answer'] = js['task7']['legend_labels_orientation']
                    qa.append(q)

            if 'task7' in js.keys():
                if 'x_label_presence' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        if js['task7']['x_label_presence']:
                            q=dict()
                            q['taxonomy id'] = '2b'
                            q['QID'] = 6
                            st = "How many <plural form of X title> are there in the chart?"
                            q['question'] = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                            q['answer'] = js['task7']['number_of_x_ticks']
                            qa.append(q)
                            
            if 'task7' in js.keys():
                if 'number_of_x_ticks' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines' or js['task7']['figure_type'] == 'scatter points':
                        q=dict()
                        q['taxonomy id'] = '2b'
                        q['QID'] = '6a'
                        q['question'] = "How many major ticks are there on the independent axis of the chart?"
                        q['answer'] = js['task7']['number_of_x_ticks']
                        qa.append(q)
                        
            if 'task7' in js.keys():
                if 'x_ticks_from_left' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '2b'
                    q['QID'] = '6b'
                    q['question'] = "List all the major ticks on independent axis from left?"
                    q['answer'] = tuple(js['task7']['x_ticks_from_left'])
                    qa.append(q)
                        
            if 'task7' in js.keys():
                if 'x_ticks_from_right' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '2b'
                    q['QID'] = '6c'
                    q['question'] = "List all the major ticks on independent axis from right?"
                    q['answer'] = tuple(js['task7']['x_ticks_from_right'])
                    qa.append(q)

            if 'task7' in js.keys():
                if 'x_ticks_from_top' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '2b'
                    q['QID'] = '6d'
                    q['question'] = "List all the major ticks on independent axis from top?"
                    q['answer'] = tuple(js['task7']['x_ticks_from_top'])
                    qa.append(q)

            if 'task7' in js.keys():
                if 'x_ticks_from_bottom' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '2b'
                    q['QID'] = '6e'
                    q['question'] = "List all the major ticks on independent axis from bottom?"
                    q['answer'] = tuple(js['task7']['x_ticks_from_bottom'])
                    qa.append(q)

            if 'task7' in js.keys():
                if 'number_of_figures' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '3a'
                    q['QID'] = 7
                    st = "How many <figure-type>s are visually present?"
                    q['question'] = st.replace("<figure-type>s",js['task7']['figure_type'])
                    q['answer'] = js['task7']['number_of_figures']
                    qa.append(q)
                    
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'legends' in js['task7'].keys():
                    if 'x_ticks_from_top' in js['task7'].keys() or 'x_ticks_from_left' in js['task7'].keys():
                        if js['task7']['figure_type'] == 'bars':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                tic_len = len(js['task7']['x_ticks_from_top'])
                            elif 'x_ticks_from_left' in js['task7'].keys():
                                tic_len = len(js['task7']['x_ticks_from_left'])
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '7b'
                            st = "How many bars are there?"
                            q['question'] = st
                            q['answer'] = len(js['task7']['legends']) * tic_len
                            qa.append(q)

            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and js['task7']['is_grouped'] == True:
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 9
                        q['question'] = "How many groups of bars are there?"
                        if 'total_number_of_grps' in js['task7'].keys():
                            q['answer'] = js['task7']['total_number_of_grps']
                            qa.append(q)

            if 'task1' in js.keys() and 'task7' in js.keys():
                if js['task1']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        q=dict()
                        q['taxonomy id'] = '2j'
                        q['QID'] = 16
                        q['question'] = "Are all the bars in the chart visually horizontal?"
                        q['answer'] = title_horizontal
                        qa.append(q)

            if 'task1' in js.keys() and 'task7' in js.keys():
                if js['task1']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        q=dict()
                        q['taxonomy id'] = '2j'
                        q['QID'] = '16a'
                        q['question'] = "Are all the bars in the chart visually vertical?"
                        q['answer'] = 'yes' if title_horizontal == 'no' else 'no'
                        qa.append(q)

            if 'task1' in js.keys() and 'task7' in js.keys():
                if js['task1']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and title_horizontal == "no":
                        st = "How many bars are there on the ith tick from the left,visually?"
                        if 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_stacked']:
                                if 'stacks_per_tick_from_left' in js['task7'].keys():
                                    for i in range(len(js['task7']['stacks_per_tick_from_left'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 12
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['stacks_per_tick_from_left'][i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys():
                            if js['task7']['is_grouped']:
                                if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_left'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 12
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_left'][i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_grouped'] == False and js['task7']['is_stacked'] == False:
                                if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_left'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 12
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_left'][i]
                                        qa.append(q)

            if 'task1' in js.keys() and 'task7' in js.keys():
                if js['task1']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and title_horizontal == "no":
                        st = "How many bars are there on the ith tick from the right,visually?"
                        if 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_stacked']:
                                if 'stacks_per_tick_from_left' in js['task7'].keys():
                                    for i in range(len(js['task7']['stacks_per_tick_from_left'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 13
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['stacks_per_tick_from_left'][len(js['task7']['stacks_per_tick_from_left'])-1-i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys():
                            if js['task7']['is_grouped']:
                                if 'number_of_grps_per_tick_from_right' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_right'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 13
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_right'][i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_grouped'] == False and js['task7']['is_stacked'] == False:
                                if 'number_of_grps_per_tick_from_right' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_right'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 13
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_right'][i]
                                        qa.append(q)

            if 'task1' in js.keys() and 'task7' in js.keys():
                if js['task1']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and title_horizontal == "yes":
                        st = "How many bars are there on the ith tick from the top,visually?"
                        if 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_stacked']:
                                if 'stacks_per_tick_from_top' in js['task7'].keys():
                                    for i in range(len(js['task7']['stacks_per_tick_from_top'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 14
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['stacks_per_tick_from_top'][i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys():
                            if js['task7']['is_grouped']:
                                if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_top'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 14
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_top'][i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_grouped'] == False and js['task7']['is_stacked'] == False:
                                if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_top'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 14
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_top'][i]
                                        qa.append(q)

            if 'task1' in js.keys() and 'task7' in js.keys():
                if js['task1']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and title_horizontal == "yes":
                        st = "How many bars are there on the ith tick from the bottom,visually?"
                        if 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_stacked']:
                                if 'stacks_per_tick_from_top' in js['task7'].keys():
                                    for i in range(len(js['task7']['stacks_per_tick_from_top'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 15
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['stacks_per_tick_from_top'][len(js['task7']['stacks_per_tick_from_top']) - 1 - i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys():
                            if js['task7']['is_grouped']:
                                if 'number_of_grps_per_tick_from_bottom' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_bottom'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 15
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_bottom'][i]
                                        qa.append(q)
                        if 'is_grouped' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                            if js['task7']['is_grouped'] == False and js['task7']['is_stacked'] == False:
                                if 'number_of_grps_per_tick_from_bottom' in js['task7'].keys():
                                    for i in range(len(js['task7']['number_of_grps_per_tick_from_bottom'])):
                                        q=dict()
                                        q['taxonomy id'] = '2j'
                                        q['QID'] = 15
                                        q['question'] = st.replace("ith",str(i+1)+add_num(i+1))
                                        q['answer'] = js['task7']['number_of_grps_per_tick_from_bottom'][i]
                                        qa.append(q)

            if 'task7' in js.keys():
                if 'number_of_figures' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        if 'legends' in js['task7'].keys():
                            q=dict()
                            q['taxonomy id'] = '2j'
                            q['QID'] = 18
                            q['question'] = "Is the number of lines equal to the number of legend labels?"
                            q['answer'] = "yes" if js['task7']['number_of_figures'] == js['task7']['legend_length'] else "no"
                            qa.append(q)

            if 'task7' in js.keys():
                if 'number_of_figures' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        if 'mark_labels_lst' in js['task7'].keys():
                            q=dict()
                            q['taxonomy id'] = '2j'
                            q['QID'] = '18a'
                            q['question'] = "Is the number of lines equal to the number of mark labels?"
                            q['answer'] = "yes" if js['task7']['number_of_figures'] == js['task7']['number_of_mark_label'] else "no"
                            qa.append(q)
                                        
            if 'task7' in js.keys() and 'task1' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task1']!=None:
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == 'no':
                            if 'is_stacked' in js['task7'].keys() or 'is_grouped' in js['task7'].keys():
                                if js['task7']['is_stacked'] or js['task7']['is_grouped']:
                                    if 'x_ticks_from_left' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_left'])>0:
                                            st = "What is the label of the jth group of bars from the left?"
                                            for i in range(len(js['task7']['x_ticks_from_left'])):
                                                q=dict()
                                                q['taxonomy id'] = '2b'
                                                q['QID'] = 23
                                                q['question'] = st.replace("jth",str(i+1)+add_num(i+1))
                                                q['answer'] = js['task7']['x_ticks_from_left'][i]
                                                qa.append(q)
                                                
            if 'task7' in js.keys() and 'task1' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task1']!=None:
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == 'yes':
                            if 'is_stacked' in js['task7'].keys() or 'is_grouped' in js['task7'].keys():
                                if js['task7']['is_stacked'] or js['task7']['is_grouped']:
                                    if 'x_ticks_from_top' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_top'])>0:
                                            st = "What is the label of the jth group of bars from the top?"
                                            for i in range(len(js['task7']['x_ticks_from_top'])):
                                                q=dict()
                                                q['taxonomy id'] = '2b'
                                                q['QID'] = 24
                                                q['question'] = st.replace("jth",str(i+1)+add_num(i+1))
                                                q['answer'] = js['task7']['x_ticks_from_top'][i]
                                                qa.append(q)
                                                
            if 'task7' in js.keys():
                if 'y_ticks_from_bottom' in js['task7'].keys():
                    if len(js['task7']['y_ticks_from_bottom'])>1:
                        q=dict()
                        q['taxonomy id'] = '2c'
                        q['QID'] = 26
                        q['question'] = 'What is the difference between two consecutive major ticks of the dependant variable?'
                        tick_lst = list()
                        ticks = js['task7']['y_ticks_from_bottom']
                        for i in ticks:
                            try:
                                tick_lst.append(float(i))
                            except:
                                pass
                        if len(tick_lst)>1:
                            diff = list()
                            for i in range(len(tick_lst)-1):
                                diff.append(abs(tick_lst[i+1]-tick_lst[i]))
                            flag = 'Linear' if all(x in [diff[0],diff[0]+1,diff[0]-1] for x in diff) else 'Logarithmic'
                            if flag == 'Linear' and len(diff)>0:
                                q['answer'] = diff[0]
                                qa.append(q)
                elif 'y_ticks_from_left' in js['task7'].keys():
                    if len(js['task7']['y_ticks_from_left'])>1:
                        q=dict()
                        q['taxonomy id'] = '2c'
                        q['QID'] = 26
                        q['question'] = 'What is the difference between two consecutive major ticks of the dependant variable?'
                        tick_lst = list()
                        ticks = js['task7']['y_ticks_from_left']
                        for i in ticks:
                            try:
                                tick_lst.append(float(i))
                            except:
                                pass
                        if len(tick_lst)!=0:
                            diff = list()
                            for i in range(len(tick_lst)-1):
                                diff.append(abs(tick_lst[i+1]-tick_lst[i]))
                            flag = 'Linear' if all(x in [diff[0],diff[0]+1,diff[0]-1] for x in diff) else 'Logarithmic'
                            if flag == 'Linear' and len(diff)>0:
                                q['answer'] = diff[0]
                                qa.append(q)
                        
            if 'task7' in js.keys():
                if 'y_ticks_from_bottom' in js['task7'].keys():
                    if len(js['task7']['y_ticks_from_bottom'])>1:
                        q=dict()
                        q['taxonomy id'] = '2c'
                        q['QID'] = 27
                        q['question'] = 'Are the values on the major ticks of dependent axis written in scientific E-notation?'
                        y_text = js['task7']['y_ticks_from_bottom']
                        ans=list()
                        for i in y_text:
                            if 'e' in i.lower():
                                ans.append("e")
                            else:
                                ans.append("n")
                        q['answer'] = "yes" if all(i=="e" for i in ans) else "no"
                        qa.append(q)
                elif 'y_ticks_from_left' in js['task7'].keys():
                    if len(js['task7']['y_ticks_from_left'])>1:
                        q=dict()
                        q['taxonomy id'] = '2c'
                        q['QID'] = 27
                        q['question'] = 'Are the values on the major ticks of dependent axis written in scientific E-notation?'
                        y_text = js['task7']['y_ticks_from_left']
                        ans=list()
                        for i in y_text:
                            if 'e' in i.lower():
                                ans.append("e")
                            else:
                                ans.append("n")
                        q['answer'] = "yes" if all(i=="e" for i in ans) else "no"
                        qa.append(q)
                        
            if 'task7' in js.keys():
                q=dict()
                q['taxonomy id'] = '1a'
                q['QID'] = '28a'
                q['question'] = 'Is there a title in the chart?'
                if 'chart_title' in js['task7'].keys():
                    q['answer'] = 'yes'
                else:
                    q['answer'] = 'no'
                qa.append(q)
                
            if 'task7' in js.keys():
                if 'chart_title' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '1a'
                    q['QID'] = 28
                    q['question'] = 'What is the title of the chart?'
                    q['answer'] = js['task7']['chart_title']
                    qa.append(q)
                    
            if 'task7' in js.keys():
                questions = list()
                answers = list()
                st = "Does <legend_label/x_label/y-label> appear as one of the legend labels in the graph?"
                if 'legends' in js['task7'].keys():
                    for i in js['task7']['legends']:
                        questions.append(st.replace("<legend_label/x_label/y-label>",i))
                        answers.append("yes")
                if 'x_label' in js['task7'].keys():
                    questions.append(st.replace("<legend_label/x_label/y-label>","'"+js['task7']['x_label']+"'"))
                    answers.append("no")
                if 'y_label' in js['task7'].keys():
                    questions.append(st.replace("<legend_label/x_label/y-label>","'"+js['task7']['y_label']+"'"))
                    answers.append("no")
                if len(questions)!=0:
                    for i in range(len(questions)):
                        q=dict()
                        q['taxonomy id'] = '2g'
                        q['QID'] = 29
                        q['question'] = questions[i]
                        q['answer'] = answers[i]
                        qa.append(q)
                        
            if 'task7' in js.keys():
                if 'x_label' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '2a'
                    q['QID'] = 30
                    q['question'] = "What is the label or title of the independent axis?"
                    q['answer'] = js['task7']['x_label']
                    qa.append(q)
                    
            if 'task7' in js.keys():
                if 'y_label' in js['task7'].keys():
                    q=dict()
                    q['taxonomy id'] = '2c'
                    q['QID'] = 31
                    q['question'] = "What is the label or title of the dependent axis?"
                    q['answer'] = js['task7']['y_label']
                    qa.append(q)
                    
            if 'task7' in js.keys() and 'task1' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task1']!=None:
                    if js['task7']['figure_type'] == 'bars':
                        st = "In how many cases, is the number of <figure_type> for a given <X_label> not equal to the number of legend labels?"
                        if 'x_label' in js['task7'].keys():
                            st = st.replace("<X_label>","'"+js['task7']['x_label']+"'")
                            q=dict()
                            q['taxonomy id'] = '2j'
                            q['QID'] = 32
                            q['question'] = st.replace("<figure_type>",js['task7']['figure_type'])
                            if 'legends' in js['task7'].keys():
                                if len(js['task7']['legends'])>0:
                                    leg = len(js['task7']['legends'])
                                    if title_horizontal == "yes":
                                        if 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                                            if js['task7']['is_grouped'] == False and js['task7']['is_stacked']:
                                                if 'stacks_per_tick_from_top' in js['task7'].keys():
                                                    if len(js['task7']['stacks_per_tick_from_top'])>0:
                                                        count = 0
                                                        for i in js['task7']['stacks_per_tick_from_top']:
                                                            if i!=leg:
                                                                count+=1
                                                        q['answer'] = count
                                                        qa.append(q)
                                        elif 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                                            if js['task7']['is_stacked'] == False and js['task7']['is_grouped']:
                                                if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                                    if len(js['task7']['number_of_grps_per_tick_from_top'])>0:
                                                        count = 0
                                                        for i in js['task7']['number_of_grps_per_tick_from_top']:
                                                            if i!=leg:
                                                                count+=1
                                                        q['answer'] = count
                                                        qa.append(q)
                                    else:
                                        if 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                                            if js['task7']['is_grouped'] == False and js['task7']['is_stacked']:
                                                if 'stacks_per_tick_from_left' in js['task7'].keys():
                                                    if len(js['task7']['stacks_per_tick_from_left'])>0:
                                                        count = 0
                                                        for i in js['task7']['stacks_per_tick_from_left']:
                                                            if i!=leg:
                                                                count+=1
                                                        q['answer'] = count
                                                        qa.append(q)
                                        elif 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                                            if js['task7']['is_stacked'] == False and js['task7']['is_grouped']:
                                                if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                                    if len(js['task7']['number_of_grps_per_tick_from_left'])>0:
                                                        count = 0
                                                        for i in js['task7']['number_of_grps_per_tick_from_left']:
                                                            if i!=leg:
                                                                count+=1
                                                        q['answer'] = count
                                                        qa.append(q)
                                                        
            if 'task7' in js.keys() and 'task1' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task1']!=None:
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == "yes":
                            if 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                                if js['task7']['is_stacked'] and js['task7']['is_grouped']==False:
                                    if 'stacks_per_tick_from_top' in js['task7'].keys():
                                        if len(js['task7']['stacks_per_tick_from_top'])!=0:
                                            q=dict()
                                            q['taxonomy id'] = '2h'
                                            q['QID'] = 10
                                            q['question'] = 'Are the number of bars on each tick equal to the number of data series?'
                                            if 'legends' in js['task7'].keys():
                                                leg = len(js['task7']['legends'])
                                            else:
                                                leg = 1
                                            flag = 0
                                            for i in js['task7']['stacks_per_tick_from_top']:
                                                if i!=leg:
                                                    flag = 1
                                                    break
                                            q['answer'] = "yes" if flag == 0 else "no"
                                            qa.append(q)
                                elif js['task7']['is_grouped'] and js['task7']['is_stacked']==False:
                                    if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                        if len(js['task7']['number_of_grps_per_tick_from_top'])!=0:
                                            q=dict()
                                            q['taxonomy id'] = '2h'
                                            q['QID'] = 10
                                            q['question'] = 'Are the number of bars on each tick equal to the number of data series?'
                                            if 'legends' in js['task7'].keys():
                                                leg = len(js['task7']['legends'])
                                            else:
                                                leg = 1
                                            flag = 0
                                            for i in js['task7']['number_of_grps_per_tick_from_top']:
                                                if i!=leg:
                                                    flag = 1
                                                    break
                                            q['answer'] = "yes" if flag == 0 else "no"
                                            qa.append(q)
                                elif js['task7']['is_grouped']==False and js['task7']['is_stacked']==False:
                                    if 'number_of_grps_per_tick_from_top' in js['task7'].keys():
                                        if len(js['task7']['number_of_grps_per_tick_from_top'])!=0:
                                            q=dict()
                                            q['taxonomy id'] = '2h'
                                            q['QID'] = 10
                                            q['question'] = 'Are the number of bars on each tick equal to the number of data series?'
                                            if 'legends' in js['task7'].keys():
                                                leg = len(js['task7']['legends'])
                                            else:
                                                leg = 1
                                            flag = 0
                                            for i in js['task7']['number_of_grps_per_tick_from_top']:
                                                if i!=leg:
                                                    flag = 1
                                                    break
                                            q['answer'] = "yes" if flag == 0 else "no"
                                            qa.append(q)
                        else:
                            if 'is_stacked' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                                if js['task7']['is_stacked'] and js['task7']['is_grouped']==False:
                                    if 'stacks_per_tick_from_left' in js['task7'].keys():
                                        if len(js['task7']['stacks_per_tick_from_left'])!=0:
                                            q=dict()
                                            q['taxonomy id'] = '2h'
                                            q['QID'] = 10
                                            q['question'] = 'Are the number of bars on each tick equal to the number of data series?'
                                            if 'legends' in js['task7'].keys():
                                                leg = len(js['task7']['legends'])
                                            else:
                                                leg = 1
                                            flag = 0
                                            for i in js['task7']['stacks_per_tick_from_left']:
                                                if i!=leg:
                                                    flag = 1
                                                    break
                                            q['answer'] = "yes" if flag == 0 else "no"
                                            qa.append(q)
                                elif js['task7']['is_grouped'] and js['task7']['is_stacked']==False:
                                    if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                        if len(js['task7']['number_of_grps_per_tick_from_left'])!=0:
                                            q=dict()
                                            q['taxonomy id'] = '2h'
                                            q['QID'] = 10
                                            q['question'] = 'Are the number of bars on each tick equal to the number of data series?'
                                            if 'legends' in js['task7'].keys():
                                                leg = len(js['task7']['legends'])
                                            else:
                                                leg = 1
                                            flag = 0
                                            for i in js['task7']['number_of_grps_per_tick_from_left']:
                                                if i!=leg:
                                                    flag = 1
                                                    break
                                            q['answer'] = "yes" if flag == 0 else "no"
                                            qa.append(q)
                                elif js['task7']['is_grouped']==False and js['task7']['is_stacked']==False:
                                    if 'number_of_grps_per_tick_from_left' in js['task7'].keys():
                                        if len(js['task7']['number_of_grps_per_tick_from_left'])!=0:
                                            q=dict()
                                            q['taxonomy id'] = '2h'
                                            q['QID'] = 10
                                            q['question'] = 'Are the number of bars on each tick equal to the number of data series?'
                                            if 'legends' in js['task7'].keys():
                                                leg = len(js['task7']['legends'])
                                            else:
                                                leg = 1
                                            flag = 0
                                            for i in js['task7']['number_of_grps_per_tick_from_left']:
                                                if i!=leg:
                                                    flag = 1
                                                    break
                                            q['answer'] = "yes" if flag == 0 else "no"
                                            qa.append(q)
                                            
            if 'task1' in js.keys():
                if js['task1']!=None:
                    q=dict()
                    q['taxonomy id'] = '1a'
                    q['QID'] = '7a'
                    q['question'] = "What is the type of chart?"
                    if 'bar' in js['task1']['output']['chart_type'].split():
                        if title_horizontal == "yes":
                            q['answer'] = 'Horizontal Bar chart'
                        else:
                            q['answer'] = 'Vertical Bar chart'
                    elif 'line' in js['task1']['output']['chart_type'].split():
                        q['answer'] = 'Line chart'
                    elif 'scatter' in js['task1']['output']['chart_type'].split():
                        q['answer'] = 'Scatter chart'
                    else:
                        q['answer'] = js['task1']['output']['chart_type']
                    qa.append(q)
                    
            if 'task7' in js.keys() and 'task1' in js.keys() and 'task6' in js.keys():
                if 'legend_presence' in js['task7'].keys() and 'figure_type' in js['task7'].keys() and js['task1']!=None and js['task6']!=None:
                    if js['task7']['legend_presence'] == False and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                        st = "What is the <Y_value> at <(ith) X tick>?"
                        st = st.replace("<Y_value>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            if title_horizontal == "yes":
                                if 'x_ticks_from_top' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_top'])!=0:
                                        for i in js['task7']['x_ticks_from_top']:
                                            q=dict()
                                            q['taxonomy id'] = '3b'
                                            q['QID'] = 33
                                            q['question'] = st.replace("<(ith) X tick>",i)
                                            for j in js['task6']['output']['data series']:
                                                for k in j['data']:
                                                    if k['x'] == i:
                                                        if 'y' in k.keys():
                                                            q['answer'] = k['y']
                                                            qa.append(q)
                                                        elif 'y2' in k.keys():
                                                            q['answer'] = k['y2']
                                                            qa.append(q)
                            else:
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])!=0:
                                        for i in js['task7']['x_ticks_from_left']:
                                            q=dict()
                                            q['taxonomy id'] = '3b'
                                            q['QID'] = 33
                                            q['question'] = st.replace("<(ith) X tick>",i)
                                            for j in js['task6']['output']['data series']:
                                                for k in j['data']:
                                                    if k['x'] == i:
                                                        if 'y' in k.keys():
                                                            q['answer'] = k['y']
                                                            qa.append(q)
                                                        elif 'y2' in k.keys():
                                                            q['answer'] = k['y2']
                                                            qa.append(q)
                        elif js['task7']['figure_type'] == 'lines':
                            if 'name_of_data_series' in js['task7'].keys():
                                if len(js['task7']['name_of_data_series'])==1:
                                    if 'extrapolated' in js['task7'].keys():
                                        for i in js['task7']['extrapolated'].keys():
                                            for j in js['task7']['extrapolated'][i]:
                                                x,y = j[0],j[1]
                                                q=dict()
                                                q['taxonomy id'] = '3b'
                                                q['QID'] = 33
                                                q['question'] = st.replace("<(ith) X tick>",str(x))
                                                q['answer'] = y
                                                qa.append(q)
                            else:
                                if 'extrapolated' in js['task7'].keys():
                                    for i in js['task7']['extrapolated'].keys():
                                        for j in js['task7']['extrapolated'][i]:
                                            x,y = j[0],j[1]
                                            q=dict()
                                            q['taxonomy id'] = '3b'
                                            q['QID'] = 33
                                            q['question'] = st.replace("<(ith) X tick>",str(x))
                                            q['answer'] = y
                                            qa.append(q)
                                            
            if 'task7' in js.keys() and 'task1' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task1']!=None and js['task6']!=None:
                    if 'y_label' in js['task7'].keys():
                        st = "What is the <Y_value> of the ith <legend_label> at < ith X_tick>?"
                        st = st.replace("<Y_value>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            if 'legends' in js['task7'].keys():
                                if title_horizontal == "yes":
                                    if 'x_ticks_from_top' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_top'])!=0:
                                            texts = js['task7']['x_ticks_from_top']
                                            legends_list = js['task7']['legends']
                                            for i in legends_list:
                                                for j in texts:
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == i:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == j.split():
                                                                    q=dict()
                                                                    q['taxonomy id'] = '3b'
                                                                    q['QID'] = 34
                                                                    qu = st.replace("the ith <legend_label>",i)
                                                                    q['question'] = qu.replace("< ith X_tick>",j)
                                                                    ans = ""
                                                                    if 'y' in k1.keys():
                                                                        ans = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        ans = k1['y2']
                                                                    if ans!="":
                                                                        q['answer'] = ans
                                                                        qa.append(q)
                                else:
                                    if 'x_ticks_from_left' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_left'])!=0:
                                            texts = js['task7']['x_ticks_from_left']
                                            legends_list = js['task7']['legends']
                                            for i in legends_list:
                                                for j in texts:
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == i:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == j.split():
                                                                    q=dict()
                                                                    q['taxonomy id'] = '3b'
                                                                    q['QID'] = 34
                                                                    qu = st.replace("the ith <legend_label>",i)
                                                                    q['question'] = qu.replace("< ith X_tick>",j)
                                                                    ans = ""
                                                                    if 'y' in k1.keys():
                                                                        ans = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        ans = k1['y2']
                                                                    if ans!="":
                                                                        q['answer'] = ans
                                                                        qa.append(q)
                        elif js['task7']['figure_type'] == 'lines':
                            if 'legends' in js['task7'].keys() or 'mark_labels_lst' in js['task7'].keys() or 'name_of_data_series' in js['task7'].keys():
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    if 'legends' in js['task7'].keys():
                                        legends_list = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legends_list = js['task7']['mark_labels_lst']
                                    elif 'name_of_data_series' in js['task7'].keys():
                                        if len(js['task7']['name_of_data_series'])>1:
                                            legends_list = js['task7']['name_of_data_series']
                                    for i in legends_list:
                                        for j in texts:
                                            for k in js['task7']['extrapolated'].keys():
                                                if k == i:
                                                    for k1 in js['task7']['extrapolated'][k]:
                                                        if float(k1[0]) == float(j):
                                                            q=dict()
                                                            q['taxonomy id'] = '3b'
                                                            q['QID'] = 34
                                                            qu = st.replace("the ith <legend_label>",i)
                                                            q['question'] = qu.replace("< ith X_tick>",j)
                                                            q['answer'] = k1[1]
                                                            qa.append(q)
                            
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type']=='lines':
                        if 'name_of_data_series' in js['task7'].keys():
                            if len(js['task7']['name_of_data_series'])==1:
                                st = 'Does the <Y title> monotonically increase over the <plural form of X title>?'
                                st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 35
                                q['question'] = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                                data_pts = list()
                                for i in js['task6']['output']['data series']:
                                    for j in i['data']:
                                        data_pts.append(j['y'])
                                f=1
                                for i in range(len(data_pts)-1):
                                    if(data_pts[i]<data_pts[i+1]):
                                        f=1
                                    else:
                                        f=0
                                        break
                                q['answer'] = "yes" if f==1 else "no"
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "Across all <plural form of X title>, what is the maximum value of <Y title>?"
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        q=dict()
                        q['taxonomy id'] = '3b'
                        q['QID'] = 38
                        q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        ans = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    ans.append(j['y'])
                                elif 'y2' in j.keys():
                                    ans.append(j['y2'])
                        q['answer'] = max(ans)
                        qa.append(q)
                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            st = "Across all major ticks on independent axis, what is the maximum value of <Y title>?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '38a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            q['answer'] = max(dat)
                            qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'extrapolated' in js['task7'].keys():
                            st = "Across all major ticks on independent axis, what is the maximum value of <Y title>?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '38a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            q['answer'] = max(dat)
                            qa.append(q)
                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "Across all <plural form of X title>, what is the minimum value of <Y title>?"
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        q=dict()
                        q['taxonomy id'] = '3b'
                        q['QID'] = 39
                        q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        ans = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    ans.append(j['y'])
                                elif 'y2' in j.keys():
                                    ans.append(j['y2'])
                        q['answer'] = min(ans)
                        qa.append(q)
                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            st = "Across all major ticks on independent axis, what is the minimum value of <Y title>?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '39a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            q['answer'] = min(dat)
                            qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'extrapolated' in js['task7'].keys():
                            st = "Across all major ticks on independent axis, what is the minimum value of <Y title>?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '39a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            q['answer'] = min(dat)
                            qa.append(q)
                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "For which value of <X title> are the <Y title> maximum?"
                        st = st.replace("<X title>","'"+js['task7']['x_label']+"'")
                        if 'y' in js['task6']['output']['data series'][0]['data'][0].keys():
                            dat = js['task6']['output']['data series'][0]['data']
                            data = sorted(dat,key=lambda x:x['y'],reverse = True)
                            max_val = data[0]['y']
                            answer = list()
                            for i in dat:
                                if i['y'] == max_val:
                                    answer.append(i['x'])
                        elif 'y2' in js['task6']['output']['data series'][0]['data'][0].keys():
                            dat = js['task6']['output']['data series'][0]['data']
                            data = sorted(dat,key=lambda x:x['y2'],reverse = True)
                            max_val = data[0]['y2']
                            answer = list()
                            for i in dat:
                                if i['y2'] == max_val:
                                    answer.append(i['x'])
                        if len(answer)!=0:
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = 40
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            q['answer'] = ",".join(answer)
                            qa.append(q)
                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            st = "For which major ticks on independent axis was the <Y title> maximum?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '40a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            key = list(js['task7']['extrapolated'].keys())
                            dat = js['task7']['extrapolated'][key[0]]
                            data = sorted(dat,key = lambda x:x[1],reverse = True)
                            max_val = data[0][1]
                            answer = list()
                            for i in data:
                                if i[1] == max_val:
                                    answer.append(i[0])
                            answe_lst = list()
                            for a in answer:
                                answe_lst.append(str(a))
                            q['answer'] = ",".join(answe_lst)
                            qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'extrapolated' in js['task7'].keys():
                            st = "For which major ticks on independent axis was the <Y title> maximum?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '40a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            key = list(js['task7']['extrapolated'].keys())
                            dat = js['task7']['extrapolated'][key[0]]
                            data = sorted(dat,key = lambda x:x[1],reverse = True)
                            max_val = data[0][1]
                            answer = list()
                            for i in data:
                                if i[1] == max_val:
                                    answer.append(i[0])
                            answe_lst = list()
                            for a in answer:
                                answe_lst.append(str(a))
                            q['answer'] = ",".join(answe_lst)
                            qa.append(q)
                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        st = "For which value of <X title> are the <Y title> minimum?"
                        st = st.replace("<X title>","'"+js['task7']['x_label']+"'")
                        if 'y' in js['task6']['output']['data series'][0]['data'][0].keys():
                            dat = js['task6']['output']['data series'][0]['data']
                            data = sorted(dat,key=lambda x:x['y'])
                            max_val = data[0]['y']
                            answer = list()
                            for i in dat:
                                if i['y'] == max_val:
                                    answer.append(i['x'])
                        elif 'y2' in js['task6']['output']['data series'][0]['data'][0].keys():
                            dat = js['task6']['output']['data series'][0]['data']
                            data = sorted(dat,key=lambda x:x['y2'])
                            max_val = data[0]['y2']
                            answer = list()
                            for i in dat:
                                if i['y2'] == max_val:
                                    answer.append(i['x'])
                        if len(answer)!=0:
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = 41
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            q['answer'] = ",".join(answer)
                            qa.append(q)
                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            st = "For which major ticks on independent axis was the <Y title> minimum?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '41a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            key = list(js['task7']['extrapolated'].keys())
                            dat = js['task7']['extrapolated'][key[0]]
                            data = sorted(dat,key = lambda x:x[1])
                            max_val = data[0][1]
                            answer = list()
                            for i in data:
                                if i[1] == max_val:
                                    answer.append(i[0])
                            answe_lst = list()
                            for a in answer:
                                answe_lst.append(str(a))
                            q['answer'] = ",".join(answe_lst)
                            qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'extrapolated' in js['task7'].keys():
                            st = "For which major ticks on independent axis was the <Y title> minimum?"
                            q=dict()
                            q['taxonomy id'] = '3b'
                            q['QID'] = '41a'
                            q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            key = list(js['task7']['extrapolated'].keys())
                            dat = js['task7']['extrapolated'][key[0]]
                            data = sorted(dat,key = lambda x:x[1])
                            max_val = data[0][1]
                            answer = list()
                            for i in data:
                                if i[1] == max_val:
                                    answer.append(i[0])
                            answe_lst = list()
                            for a in answer:
                                answe_lst.append(str(a))
                            q['answer'] = ",".join(answe_lst)
                            qa.append(q)
                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    st = "Across all <plural form of X title>, what is the maximum <Y title> for <legend label>?"
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                            for i in legends_labels:
                                dat = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name']==i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                if len(dat)>1:
                                    q=dict()
                                    q['taxonomy id'] = '3b'
                                    q['QID'] = 42
                                    q['question'] = st.replace("<legend label>",i)
                                    q['answer'] = max(dat)
                                    qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "Across all major ticks of independent axis, what is the maximum <Y title> for <legend label>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if 'mark_labels_lst' in js['task7'].keys():
                            legends_labels = js['task7']['mark_labels_lst']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1:
                            if 'extrapolated' in js['task7'].keys():
                                for i in legends_labels:
                                    dat = list()
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                    if len(dat)>1:
                                        q=dict()
                                        q['taxonomy id'] = '3b'
                                        q['QID'] = '42a'
                                        q['question'] = st.replace("<legend label>",i)
                                        q['answer'] = max(dat)
                                        qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        legends_labels = list()
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1:
                            if 'extrapolated' in js['task7'].keys():
                                for i in legends_labels:
                                    dat = list()
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                    if len(dat)>1:
                                        q=dict()
                                        q['taxonomy id'] = '3b'
                                        q['QID'] = '42a'
                                        q['question'] = st.replace("<legend label>",i)
                                        q['answer'] = max(dat)
                                        qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    st = "Across all <plural form of X title>, what is the minimum <Y title> for <legend label>?"
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                            for i in legends_labels:
                                dat = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name']==i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                if len(dat)>1:
                                    q=dict()
                                    q['taxonomy id'] = '3b'
                                    q['QID'] = 43
                                    q['question'] = st.replace("<legend label>",i)
                                    q['answer'] = min(dat)
                                    qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "Across all major ticks on independent axis, what is the minimum <Y title> for <legend label>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if 'mark_labels_lst' in js['task7'].keys():
                            legends_labels = js['task7']['mark_labels_lst']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1:
                            if 'extrapolated' in js['task7'].keys():
                                for i in legends_labels:
                                    dat = list()
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                    if len(dat)>1:
                                        q=dict()
                                        q['taxonomy id'] = '3b'
                                        q['QID'] = '43a'
                                        q['question'] = st.replace("<legend label>",i)
                                        q['answer'] = min(dat)
                                        qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1:
                            if 'extrapolated' in js['task7'].keys():
                                for i in legends_labels:
                                    dat = list()
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                    if len(dat)>1:
                                        q=dict()
                                        q['taxonomy id'] = '3b'
                                        q['QID'] = '43a'
                                        q['question'] = st.replace("<legend label>",i)
                                        q['answer'] = min(dat)
                                        qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    st = "For which value of <singular form of X title> was the <Y title> for <legend label> maximum?"
                    st = st.replace("<singular form of X title>","'"+js['task7']['x_label']+"'")
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            lst_questions = list()   # all combination of questions
                            for i in legend_labels:
                                for j in js['task6']['output']['data series']:
                                    k = j['data']
                                    if 'y' in k[0].keys():
                                        ans = sorted(k,key=lambda x:x['y'],reverse=True)
                                        answer_lst = list()
                                        for a in ans:
                                            if a['y'] == ans[0]['y']:
                                                answer_lst.append(a['x'])
                                    elif 'y2' in k[0].keys():
                                        ans = sorted(k,key=lambda x:x['y2'],reverse=True)
                                        answer_lst = list()
                                        for a in ans:
                                            if a['y2'] == ans[0]['y2']:
                                                answer_lst.append(a['x'])
                                q=dict()
                                q['taxonomy id'] = '3b'
                                q['QID'] = 44
                                q['question'] = st.replace("<legend label>",i)
                                q['answer'] = ",".join(answer_lst)
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "For which major tick on independent axis, was the <Y title> for <legend label> maximum?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if 'mark_labels_lst' in js['task7'].keys():
                            legends_labels = js['task7']['mark_labels_lst']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1 and 'extrapolated' in js['task7'].keys():
                            for i in legends_labels:
                                if i in js['task7']['extrapolated'].keys():
                                    dat = js['task7']['extrapolated'][i]
                                    data = sorted(dat,key = lambda x:x[1],reverse = True)
                                    answer=list()
                                    for k in data:
                                        if k[1]==data[0][1]:
                                            answer.append(k[0])
                                    q=dict()
                                    q['taxonomy id'] = '3b'
                                    q['QID'] = '44a'
                                    q['question'] = st.replace("<legend label>",i)
                                    answe_lst = list()
                                    for a in answer:
                                        answe_lst.append(str(a))
                                    q['answer'] = ",".join(answe_lst)
                                    qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1 and 'extrapolated' in js['task7'].keys():
                            for i in legends_labels:
                                if i in js['task7']['extrapolated'].keys():
                                    dat = js['task7']['extrapolated'][i]
                                    data = sorted(dat,key = lambda x:x[1],reverse = True)
                                    answer=list()
                                    for k in data:
                                        if k[1]==data[0][1]:
                                            answer.append(k[0])
                                    q=dict()
                                    q['taxonomy id'] = '3b'
                                    q['QID'] = '44a'
                                    q['question'] = st.replace("<legend label>",i)
                                    answe_lst = list()
                                    for a in answer:
                                        answe_lst.append(str(a))
                                    q['answer'] = ",".join(answe_lst)
                                    qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    st = "For which value of <singular form of X title> was the <Y title> for <legend label> minimum?"
                    st = st.replace("<singular form of X title>","'"+js['task7']['x_label']+"'")
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            lst_questions = list()   # all combination of questions
                            for i in legend_labels:
                                for j in js['task6']['output']['data series']:
                                    k = j['data']
                                    if 'y' in k[0].keys():
                                        ans = sorted(k,key=lambda x:x['y'])
                                        answer_lst = list()
                                        for a in ans:
                                            if a['y'] == ans[0]['y']:
                                                answer_lst.append(a['x'])
                                    elif 'y2' in k[0].keys():
                                        ans = sorted(k,key=lambda x:x['y2'])
                                        answer_lst = list()
                                        for a in ans:
                                            if a['y2'] == ans[0]['y2']:
                                                answer_lst.append(a['x'])
                                q=dict()
                                q['taxonomy id'] = '3b'
                                q['QID'] = 45
                                q['question'] = st.replace("<legend label>",i)
                                q['answer'] = ",".join(answer_lst)
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "For which major tick on independent axis, was the <Y title> for <legend label> minimum?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if 'mark_labels_lst' in js['task7'].keys():
                            legends_labels = js['task7']['mark_labels_lst']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1 and 'extrapolated' in js['task7'].keys():
                            for i in legends_labels:
                                if i in js['task7']['extrapolated'].keys():
                                    dat = js['task7']['extrapolated'][i]
                                    data = sorted(dat,key = lambda x:x[1])
                                    answer=list()
                                    for k in data:
                                        if k[1]==data[0][1]:
                                            answer.append(k[0])
                                    q=dict()
                                    q['taxonomy id'] = '3b'
                                    q['QID'] = '45a'
                                    q['question'] = st.replace("<legend label>",i)
                                    answe_lst = list()
                                    for a in answer:
                                        answe_lst.append(str(a))
                                    q['answer'] = ",".join(answe_lst)
                                    qa.append(q)
                    elif js['task7']['figure_type'] == 'scatter points':
                        if 'legends' in js['task7'].keys():
                            legends_labels = js['task7']['legends']
                        if len(js['task7']['name_of_data_series'])>1:
                            legends_labels = js['task7']['name_of_data_series']
                        if len(legends_labels)>1 and 'extrapolated' in js['task7'].keys():
                            for i in legends_labels:
                                if i in js['task7']['extrapolated'].keys():
                                    dat = js['task7']['extrapolated'][i]
                                    data = sorted(dat,key = lambda x:x[1])
                                    answer=list()
                                    for k in data:
                                        if k[1]==data[0][1]:
                                            answer.append(k[0])
                                    q=dict()
                                    q['taxonomy id'] = '3b'
                                    q['QID'] = '45a'
                                    q['question'] = st.replace("<legend label>",i)
                                    answe_lst = list()
                                    for a in answer:
                                        answe_lst.append(str(a))
                                    q['answer'] = ",".join(answe_lst)
                                    qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "What is the sum of <Y-title>?"
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 46
                        q['question'] = st.replace("<Y-title>","'"+js['task7']['y_label']+"'")
                        s = 0
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    s+=j['y']
                                elif 'y2' in j.keys():
                                    s+=j['y2']
                        q['answer'] = s
                        qa.append(q)
                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "What is the average <Y title> per <singular form of X title>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 48
                        q['question'] = st.replace("<singular form of X title>","'"+js['task7']['x_label']+"'")
                        dat = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat.append(j['y2'])
                        q['answer'] = np.mean(dat)
                        qa.append(q)
                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "What is the median <Y title>?"
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 49
                        q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        dat = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat.append(j['y2'])
                        q['answer'] = np.median(dat)
                        qa.append(q)
                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        st = "What is the total <Y title> for <legend label> in the graph?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        leg = js['task7']['legends']
                        for i in leg:
                            dat = list()
                            for j in js['task6']['output']['data series']:
                                if j['name']==i:
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            dat.append(k['y'])
                                        elif 'y2' in k.keys():
                                            dat.append(k['y2'])
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 50
                            q['question'] = st.replace("<legend label>",i)
                            q['answer'] = sum(dat)
                            qa.append(q)
                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "What is the difference between the value of <Y title> at < ithx tick> and < jthx tick> of <X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    questions = list()
                                    stack_lst = list()
                                    answers = list()
                                    for i in range(len(texts)-1):
                                        for j in range(i+1,len(texts)):
                                            qu = st.replace("< ithx tick>",texts[i])
                                            questions.append(qu.replace("< jthx tick>",texts[j]))
                                            stack_lst.append([texts[i],texts[j]])
                                    for i in range(len(stack_lst)):
                                        t1 = stack_lst[i][0]
                                        t2 = stack_lst[i][1]
                                        for j in js['task6']['output']['data series']:
                                            for k in j['data']:
                                                if k['x']==t1:
                                                    if 'y' in k.keys():
                                                        val1 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val1 = k['y']
                                                elif k['x']==t2:
                                                    if 'y' in k.keys():
                                                        val2 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val2 = k['y']
                                        answers.append(abs(val1-val2))
                                    for i in range(len(questions)):
                                        q = dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 47
                                        q['question'] = questions[i]
                                        q['answer'] = answers[i]
                                        qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    questions = list()
                                    stack_lst = list()
                                    answers = list()
                                    for i in range(len(texts)-1):
                                        for j in range(i+1,len(texts)):
                                            qu = st.replace("< ithx tick>",texts[i])
                                            questions.append(qu.replace("< jthx tick>",texts[j]))
                                            stack_lst.append([texts[i],texts[j]])
                                    for i in range(len(stack_lst)):
                                        t1 = stack_lst[i][0]
                                        t2 = stack_lst[i][1]
                                        for j in js['task6']['output']['data series']:
                                            for k in j['data']:
                                                if k['x']==t1:
                                                    if 'y' in k.keys():
                                                        val1 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val1 = k['y']
                                                elif k['x']==t2:
                                                    if 'y' in k.keys():
                                                        val2 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val2 = k['y']
                                        answers.append(abs(val1-val2))
                                    for i in range(len(questions)):
                                        q = dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 47
                                        q['question'] = questions[i]
                                        q['answer'] = answers[i]
                                        qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    questions = list()
                                    stack_lst = list()
                                    answers = list()
                                    for i in range(len(texts)-1):
                                        for j in range(i+1,len(texts)):
                                            qu = st.replace("< ithx tick>",texts[i])
                                            questions.append(qu.replace("< jthx tick>",texts[j]))
                                            stack_lst.append([texts[i],texts[j]])
                                    for i in range(len(stack_lst)):
                                        t1 = float(stack_lst[i][0])
                                        t2 = float(stack_lst[i][1])
                                        for j in js['task7']['extrapolated'].keys():
                                            for k in js['task7']['extrapolated'][j]:
                                                if float(k[0])==float(t1):
                                                    val1 = k[1]
                                                elif float(k[0])==float(t2):
                                                    val2 = k[1]
                                        answers.append(abs(val1-val2))
                                    if len(answers)!=0:
                                        for i in range(len(questions)):
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = 47
                                            q['question'] = questions[i]
                                            q['answer'] = answers[i]
                                            qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "What is the difference between the value of <Y title> for <legend label> at < ithx tick> and that at <jthx tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars' and 'legends' in js['task7'].keys():
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    legends = js['task7']['legends']
                                    ticks = js['task7']['x_ticks_from_top']
                                    for i in legends:
                                        for j in range(len(ticks)-1):
                                            for k in range(j+1,len(ticks)):
                                                qu = st.replace("< ithx tick>",ticks[j])
                                                qu = qu.replace("<jthx tick>",ticks[k])
                                                for i1 in js['task6']['output']['data series']:
                                                    if i1['name'] == i:
                                                        for i2 in i1['data']:
                                                            if i2['x'].split() == ticks[j].split():
                                                                if 'y' in i2.keys():
                                                                    val1 = i2['y']
                                                                elif 'y2' in i2.keys():
                                                                    val1 = i2['y2']
                                                            elif i2['x'].split() == ticks[k].split():
                                                                if 'y' in i2.keys():
                                                                    val2 = i2['y']
                                                                elif 'y2' in i2.keys():
                                                                    val2 = i2['y2']
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = 51
                                                q['question'] = qu.replace("<legend label>",i)
                                                q['answer'] = abs(val1-val2)
                                                qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys() and 'legends' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    legends = js['task7']['legends']
                                    ticks = js['task7']['x_ticks_from_left']
                                    for i in legends:
                                        for j in range(len(ticks)-1):
                                            for k in range(j+1,len(ticks)):
                                                qu = st.replace("< ithx tick>",ticks[j])
                                                qu = qu.replace("<jthx tick>",ticks[k])
                                                for i1 in js['task6']['output']['data series']:
                                                    if i1['name'] == i:
                                                        for i2 in i1['data']:
                                                            if i2['x'].split() == ticks[j].split():
                                                                if 'y' in i2.keys():
                                                                    val1 = i2['y']
                                                                elif 'y2' in i2.keys():
                                                                    val1 = i2['y2']
                                                            elif i2['x'].split() == ticks[k].split():
                                                                if 'y' in i2.keys():
                                                                    val2 = i2['y']
                                                                elif 'y2' in i2.keys():
                                                                    val2 = i2['y2']
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = 51
                                                q['question'] = qu.replace("<legend label>",i)
                                                q['answer'] = abs(val1-val2)
                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    if 'legends' in js['task7'].keys():
                                        legends = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legends = js['task7']['mark_labels_lst']
                                    elif len(js['task7']['name_of_data_series'])>1:
                                        legends = js['task7']['name_of_data_series']
                                    ticks = js['task7']['x_ticks_from_left']
                                    for i in legends:
                                        for j in range(len(ticks)-1):
                                            for k in range(j+1,len(ticks)):
                                                qu = st.replace("< ithx tick>",ticks[j])
                                                qu = qu.replace("<jthx tick>",ticks[k])
                                                for i1 in js['task7']['extrapolated'].keys():
                                                    if i1 == i:
                                                        for i2 in js['task7']['extrapolated'][i1]:
                                                            if float(i2[0]) == float(ticks[j]):
                                                                val1 = i2[1]
                                                            elif float(i2[0]) == float(ticks[k]):
                                                                val2 = i2[1]
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = 51
                                                q['question'] = qu.replace("<legend label>",i)
                                                q['answer'] = abs(val1-val2)
                                                qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "What is the difference between the value of <Y title> for <legend label1> at < ithx tick> and the value of <Y title> for <legend label2> at <jthx tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys() and 'legends' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0 and len(js['task7']['legends'])>1:
                                    legends = js['task7']['legends']
                                    ticks = js['task7']['x_ticks_from_top']
                                    if len(legends)>1:
                                        for i in range(len(legends)-1):
                                            for i3 in range(i+1,len(legends)):
                                                for j in range(len(ticks)):
                                                    for k in range(len(ticks)):
                                                        qu = st.replace("< ithx tick>",ticks[j])
                                                        qu = qu.replace("<jthx tick>",ticks[k])
                                                        for i1 in js['task6']['output']['data series']:
                                                            if i1['name'] == legends[i]:
                                                                for i2 in i1['data']:
                                                                    if i2['x'].split() == ticks[j].split():
                                                                        if 'y' in i2.keys():
                                                                            val1 = i2['y']
                                                                        elif 'y2' in i2.keys():
                                                                            val1 = i2['y2']
                                                            elif i1['name'] == legends[i3]:
                                                                for i2 in i1['data']:
                                                                    if i2['x'].split() == ticks[k].split():
                                                                        if 'y' in i2.keys():
                                                                            val2 = i2['y']
                                                                        elif 'y2' in i2.keys():
                                                                            val2 = i2['y2']
                                                        q=dict()
                                                        q['taxonomy id'] = '4b'
                                                        q['QID'] = 52
                                                        qu = qu.replace("<legend label1>",legends[i])
                                                        q['question'] = qu.replace("<legend label2>",legends[i3])
                                                        q['answer'] = abs(val1-val2)
                                                        qa.append(q)
                        else:
                             if 'x_ticks_from_left' in js['task7'].keys() and 'legends' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0 and len(js['task7']['legends'])>1:
                                    legends = js['task7']['legends']
                                    ticks = js['task7']['x_ticks_from_left']
                                    if len(legends)>1:
                                        for i in range(len(legends)-1):
                                            for i3 in range(i+1,len(legends)):
                                                for j in range(len(ticks)):
                                                    for k in range(len(ticks)):
                                                        qu = st.replace("< ithx tick>",ticks[j])
                                                        qu = qu.replace("<jthx tick>",ticks[k])
                                                        for i1 in js['task6']['output']['data series']:
                                                            if i1['name'] == legends[i]:
                                                                for i2 in i1['data']:
                                                                    if i2['x'].split() == ticks[j].split():
                                                                        if 'y' in i2.keys():
                                                                            val1 = i2['y']
                                                                        elif 'y2' in i2.keys():
                                                                            val1 = i2['y2']
                                                            elif i1['name'] == legends[i3]:
                                                                for i2 in i1['data']:
                                                                    if i2['x'].split() == ticks[k].split():
                                                                        if 'y' in i2.keys():
                                                                            val2 = i2['y']
                                                                        elif 'y2' in i2.keys():
                                                                            val2 = i2['y2']
                                                        q=dict()
                                                        q['taxonomy id'] = '4b'
                                                        q['QID'] = 52
                                                        qu = qu.replace("<legend label1>",legends[i])
                                                        q['question'] = qu.replace("<legend label2>",legends[i3])
                                                        q['answer'] = abs(val1-val2)
                                                        qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    if 'legends' in js['task7'].keys():
                                        legends = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legends = js['task7']['mark_labels_lst']
                                    elif len(js['task7']['name_of_data_series'])>1:
                                        legends = js['task7']['name_of_data_series']
                                    ticks = js['task7']['x_ticks_from_left']
                                    if len(legends)>1:
                                        for i in range(len(legends)-1):
                                            for i3 in range(i+1,len(legends)):
                                                for j in range(len(ticks)):
                                                    for k in range(len(ticks)):
                                                        qu = st.replace("< ithx tick>",ticks[j])
                                                        qu = qu.replace("<jthx tick>",ticks[k])
                                                        for i1 in js['task7']['extrapolated'].keys():
                                                            if i1 == legends[i]:
                                                                for i2 in js['task7']['extrapolated'][i1]:
                                                                    if float(i2[0]) == float(ticks[j]):
                                                                        val1 = i2[1]
                                                            elif i1 == legends[i3]:
                                                                for i2 in js['task7']['extrapolated'][i1]:
                                                                    if float(i2[0]) == float(ticks[k]):
                                                                        val2 = i2[1]
                                                        q=dict()
                                                        q['taxonomy id'] = '4b'
                                                        q['QID'] = 52
                                                        qu = qu.replace("<legend label1>",legends[i])
                                                        q['question'] = qu.replace("<legend label2>",legends[i3])
                                                        q['answer'] = abs(val1-val2)
                                                        qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    st = "What is the average <Y title> for <legend label> per <singular form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<singular form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            leg = js['task7']['legends']
                            for i in leg:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 53
                                dat = list()
                                q['question'] = st.replace("<legend label>",i)
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            dat.append(k['y'])
                                if len(dat)!=0:
                                    q['answer'] = np.mean(dat)
                                    qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "What is the difference between the <Y title> for <legend label1> and <Y title> for <legend label2> at < ithx tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys() and 'legends' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    legends = js['task7']['legends']
                                    ticks = js['task7']['x_ticks_from_top']
                                    if len(legends)>1:
                                        for i in range(len(legends)-1):
                                            for j in range(i+1,len(legends)):
                                                for j1 in ticks:
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == legends[i]:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == j1.split():
                                                                    if 'y' in k1.keys():
                                                                        val1 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val1 = k1['y2']
                                                        elif k['name'] == legends[j]:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == j1.split():
                                                                    if 'y' in k1.keys():
                                                                        val2 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val2 = k1['y2']
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = 55
                                                    qu = st.replace("< ithx tick>",j1)
                                                    qu = qu.replace("<legend label1>",legends[i])
                                                    q['question'] = qu.replace("<legend label2>",legends[j])
                                                    q['answer'] = abs(val1 - val2)
                                                    qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys() and 'legends' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    legends = js['task7']['legends']
                                    ticks = js['task7']['x_ticks_from_left']
                                    if len(legends)>1:
                                        for i in range(len(legends)-1):
                                            for j in range(i+1,len(legends)):
                                                for j1 in ticks:
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == legends[i]:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == j1.split():
                                                                    if 'y' in k1.keys():
                                                                        val1 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val1 = k1['y2']
                                                        elif k['name'] == legends[j]:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == j1.split():
                                                                    if 'y' in k1.keys():
                                                                        val2 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val2 = k1['y2']
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = 55
                                                    qu = st.replace("< ithx tick>",j1)
                                                    qu = qu.replace("<legend label1>",legends[i])
                                                    q['question'] = qu.replace("<legend label2>",legends[j])
                                                    q['answer'] = abs(val1 - val2)
                                                    qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    if 'legends' in js['task7'].keys():
                                        legends = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legends = js['task7']['mark_labels_lst']
                                    elif len(js['task7']['name_of_data_series'])>1:
                                        legends = js['task7']['name_of_data_series']
                                    ticks = js['task7']['x_ticks_from_left']
                                    if len(legends)>1:
                                        for i in range(len(legends)-1):
                                            for j in range(i+1,len(legends)):
                                                for j1 in ticks:
                                                    for k in js['task7']['extrapolated'].keys():
                                                        if k == legends[i]:
                                                            for k1 in js['task7']['extrapolated'][k]:
                                                                if float(k1[0]) == float(j1):
                                                                    val1 = k1[1]
                                                        elif k == legends[j]:
                                                            for k1 in js['task7']['extrapolated'][k]:
                                                                if float(k1[0]) == float(j1):
                                                                    val2 = k1[1]
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = 55
                                                    qu = st.replace("< ithx tick>",j1)
                                                    qu = qu.replace("<legend label1>",legends[i])
                                                    q['question'] = qu.replace("<legend label2>",legends[j])
                                                    q['answer'] = abs(val1 - val2)
                                                    qa.append(q)
                                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars' and len(js['task7']['name_of_data_series'])==1:
                        st = "In how many <plural form of X title>, is the <Y title> greater than <N> units?"
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            dat = list()
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    dat.append(j['y'])
                            if len(dat)>1:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 56
                                N = random.uniform(min(dat),max(dat))
                                q['question'] = st.replace("<N> units",str(N))
                                answer = 0
                                for i in js['task6']['output']['data series']:
                                    for j in i['data']:
                                        if j['y']>N:
                                            answer+=1
                                q['answer'] = answer
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars' and len(js['task7']['name_of_data_series'])==1:
                        st = "What <plural form of X title> have the <Y title> greater than <N> units?"
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            dat = list()
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    dat.append(j['y'])
                            if len(dat)>1:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = '56b'
                                N = random.uniform(min(dat),max(dat))
                                q['question'] = st.replace("<N> units",str(N))
                                answer = list()
                                for i in js['task6']['output']['data series']:
                                    for j in i['data']:
                                        if j['y']>N:
                                            answer.append(j['x'])
                                q['answer'] = tuple(answer)
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars' and len(js['task7']['name_of_data_series'])==1:
                        st = "What <plural form of X title> have the value of <Y title> greater than <N> units, arrranged in increasing order of difference between value of <Y title> and <N> units?"
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            dat = list()
                            for i in js['task6']['output']['data series']:
                                for j in i['data']:
                                    dat.append(j['y'])
                            if len(dat)>1:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = '56d'
                                N = random.uniform(min(dat),max(dat))
                                q['question'] = st.replace("<N> units",str(N))
                                answer_temp = list()
                                for i in js['task6']['output']['data series']:
                                    for j in i['data']:
                                        if j['y']>N:
                                            answer_temp.append([j['x'],abs(j['y']-N)])
                                answer_temp = sorted(answer_temp,key=lambda x:x[1])
                                answer_final = list()
                                for i in answer_temp:
                                    answer_final.append(i[0])
                                q['answer'] = tuple(answer_final)
                                qa.append(q)
                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            if len(dat)>1:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = '56a'
                                N = random.uniform(min(dat),max(dat))
                                st = "For how many major tick on independent axis, is the value of <Y title> greater than <N> units?"
                                st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                                q['question'] = st.replace("<N> units",str(N))
                                answer = 0
                                for i in js['task7']['extrapolated'].keys():
                                    for j in js['task7']['extrapolated'][i]:
                                        if j[1]>N:
                                            answer+=1
                                q['answer'] = answer
                                qa.append(q)
                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            if len(dat)>1:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = '56c'
                                N = random.uniform(min(dat),max(dat))
                                st = "For what major ticks on independent axis, is the value of <Y title> greater than <N> units?"
                                st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                                q['question'] = st.replace("<N> units",str(N))
                                answer = list()
                                for i in js['task7']['extrapolated'].keys():
                                    for j in js['task7']['extrapolated'][i]:
                                        if j[1]>N:
                                            answer.append(j[0])
                                q['answer'] = tuple(answer)
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            if len(dat)>1:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = '56e'
                                N = random.uniform(min(dat),max(dat))
                                st = "What major ticks on independent axis have the value of <Y title> greater than <N> units, arranged in increasing order of difference between value of <Y title> and <N> units?"
                                st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                                q['question'] = st.replace("<N> units",str(N))
                                answer_temp = list()
                                for i in js['task7']['extrapolated'].keys():
                                    for j in js['task7']['extrapolated'][i]:
                                        if j[1]>N:
                                            answer_temp.append([j[0],abs(j[1]-N)])
                                answer_temp = sorted(answer_temp,key = lambda x:x[1])
                                answer = list()
                                for i in answer_temp:
                                    answer.append(i[0])
                                q['answer'] = tuple(answer)
                                qa.append(q)
            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "What is the ratio of the values of <Y title> at < ith x tick> to that at < jth x tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == "yes":
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    ticks_label = js['task7']['x_ticks_from_top']
                                    for i in range(len(ticks_label)-1):
                                        for j in range(i+1,len(ticks_label)):
                                            for k in js['task6']['output']['data series']:
                                                for k1 in k['data']:
                                                    if k1['x'].split() == ticks_label[i].split():
                                                        if 'y' in k1.keys():
                                                            val1 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val1 = k1['y2']
                                                    elif k1['x'].split() == ticks_label[j].split():
                                                        if 'y' in k1.keys():
                                                            val2 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val2 = k1['y2']
                                            if val2!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = 58
                                                qu = st.replace("< ith x tick>",ticks_label[i])
                                                q['question'] = qu.replace("< jth x tick>",ticks_label[j])
                                                q['answer'] = val1/val2
                                                qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    ticks_label = js['task7']['x_ticks_from_left']
                                    for i in range(len(ticks_label)-1):
                                        for j in range(i+1,len(ticks_label)):
                                            for k in js['task6']['output']['data series']:
                                                for k1 in k['data']:
                                                    if k1['x'].split() == ticks_label[i].split():
                                                        if 'y' in k1.keys():
                                                            val1 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val1 = k1['y2']
                                                    elif k1['x'].split() == ticks_label[j].split():
                                                        if 'y' in k1.keys():
                                                            val2 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val2 = k1['y2']
                                            if val2!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = 58
                                                qu = st.replace("< ith x tick>",ticks_label[i])
                                                q['question'] = qu.replace("< jth x tick>",ticks_label[j])
                                                q['answer'] = val1/val2
                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    ticks_label = js['task7']['x_ticks_from_left']
                                    for i in range(len(ticks_label)-1):
                                        for j in range(i+1,len(ticks_label)):
                                            for k in js['task7']['extrapolated'].keys():
                                                for k1 in js['task7']['extrapolated'][k]:
                                                    if float(k1[0]) == float(ticks_label[i]):
                                                        val1 = k1[1]
                                                    elif float(k1[0]) == float(ticks_label[j]):
                                                        val2 = k1[1]
                                            if val2!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = 58
                                                qu = st.replace("< ith x tick>",ticks_label[i])
                                                q['question'] = qu.replace("< jth x tick>",ticks_label[j])
                                                q['answer'] = val1/val2
                                                qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "Is the value of <Y title> at < ith x tick> less than that at < jth x tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == "yes":
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    ticks_label = js['task7']['x_ticks_from_top']
                                    for i in range(len(ticks_label)-1):
                                        for j in range(i+1,len(ticks_label)):
                                            for k in js['task6']['output']['data series']:
                                                for k1 in k['data']:
                                                    if k1['x'].split() == ticks_label[i].split():
                                                        if 'y' in k1.keys():
                                                            val1 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val1 = k1['y2']
                                                    elif k1['x'].split() == ticks_label[j].split():
                                                        if 'y' in k1.keys():
                                                            val2 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val2 = k1['y2']
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = 59
                                            qu = st.replace("< ith x tick>",ticks_label[i])
                                            q['question'] = qu.replace("< jth x tick>",ticks_label[j])
                                            q['answer'] = 'yes' if val1<val2 else 'no'
                                            qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    ticks_label = js['task7']['x_ticks_from_left']
                                    for i in range(len(ticks_label)-1):
                                        for j in range(i+1,len(ticks_label)):
                                            for k in js['task6']['output']['data series']:
                                                for k1 in k['data']:
                                                    if k1['x'].split() == ticks_label[i].split():
                                                        if 'y' in k1.keys():
                                                            val1 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val1 = k1['y2']
                                                    elif k1['x'].split() == ticks_label[j].split():
                                                        if 'y' in k1.keys():
                                                            val2 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val2 = k1['y2']
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = 59
                                            qu = st.replace("< ith x tick>",ticks_label[i])
                                            q['question'] = qu.replace("< jth x tick>",ticks_label[j])
                                            q['answer'] = 'yes' if val1<val2 else 'no'
                                            qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    ticks_label = js['task7']['x_ticks_from_left']
                                    for i in range(len(ticks_label)-1):
                                        for j in range(i+1,len(ticks_label)):
                                            for k in js['task7']['extrapolated'].keys():
                                                for k1 in js['task7']['extrapolated'][k]:
                                                    if float(k1[0]) == float(ticks_label[i]):
                                                        val1 = k1[1]
                                                    elif float(k1[0]) == float(ticks_label[j]):
                                                        val2 = k1[1]
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = 59
                                            qu = st.replace("< ith x tick>",ticks_label[i])
                                            q['question'] = qu.replace("< jth x tick>",ticks_label[j])
                                            q['answer'] = 'yes' if val1<val2 else 'no'
                                            qa.append(q)

            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "Which major ticks on independent axis has the value of <Y title> less than that at < ith x tick>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    for t in texts:
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if j['x'].split() == t.split():
                                                    if 'y' in j.keys():
                                                        val1 = j['y']
                                                    elif 'y2' in j.keys():
                                                        val1 = j['y2']
                                        ans = list()
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if 'y' in j.keys():
                                                    if j['y']<val1:
                                                        ans.append(j['x'])
                                                elif 'y2' in j.keys():
                                                    if j['y2']<val1:
                                                        ans.append(j['x'])
                                        if len(ans)!=0:
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '59a'
                                            q['question'] = st.replace("< ith x tick>",t)
                                            q['answer'] = ans
                                            qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    for t in texts:
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if j['x'].split() == t.split():
                                                    if 'y' in j.keys():
                                                        val1 = j['y']
                                                    elif 'y2' in j.keys():
                                                        val1 = j['y2']
                                        ans = list()
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if 'y' in j.keys():
                                                    if j['y']<val1:
                                                        ans.append(j['x'])
                                                elif 'y2' in j.keys():
                                                    if j['y2']<val1:
                                                        ans.append(j['x'])
                                        if len(ans)!=0:
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '59a'
                                            q['question'] = st.replace("< ith x tick>",t)
                                            q['answer'] = ans
                                            qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        st = "Which major ticks on independent axis has the value of <Y title> less than that at < ith x tick>?"
                        st = st.replace("<Y title>",'"'+js['task7']['y_label']+'"')
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    for t in texts:
                                        for i in js['task7']['extrapolated'].keys():
                                            for j in js['task7']['extrapolated'][i]:
                                                if float(j[0]) == float(t):
                                                    val1 = j[1]
                                        ans = list()
                                        for i in js['task7']['extrapolated'].keys():
                                            for j in js['task7']['extrapolated'][i]:
                                                if j[1]<val1:
                                                    ans.append(str(j[0]))
                                        if len(ans)!=0:
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '59a'
                                            q['question'] = st.replace("< ith x tick>",t)
                                            q['answer'] = ans
                                            qa.append(q)
                                            
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "Which major ticks on independent axis has the value of <Y title> less than that at < ith x tick>, arranged in the increasing order of difference between their <Y title> values?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    for t in texts:
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if j['x'].split() == t.split():
                                                    if 'y' in j.keys():
                                                        val1 = j['y']
                                                    elif 'y2' in j.keys():
                                                        val1 = j['y2']
                                        ans = list()
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if 'y' in j.keys():
                                                    if j['y']<val1:
                                                        ans.append((j['x'],abs(j['y']-val1)))
                                                elif 'y2' in j.keys():
                                                    if j['y2']<val1:
                                                        ans.append((j['x'],abs(j['y2']-val1)))
                                        ans_temp = sorted(ans,key=lambda x:x[1])
                                        final_ans = list()
                                        for a in ans_temp:
                                            final_ans.append(a[0])
                                        if len(final_ans)!=0:
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '59b'
                                            q['question'] = st.replace("< ith x tick>",t)
                                            q['answer'] = final_ans
                                            qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    for t in texts:
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if j['x'].split() == t.split():
                                                    if 'y' in j.keys():
                                                        val1 = j['y']
                                                    elif 'y2' in j.keys():
                                                        val1 = j['y2']
                                        ans = list()
                                        for i in js['task6']['output']['data series']:
                                            for j in i['data']:
                                                if 'y' in j.keys():
                                                    if j['y']<val1:
                                                        ans.append((j['x'],abs(j['y']-val1)))
                                                elif 'y2' in j.keys():
                                                    if j['y2']<val1:
                                                        ans.append((j['x'],abs(j['y2']-val1)))
                                        ans_temp = sorted(ans,key=lambda x:x[1])
                                        final_ans = list()
                                        for a in ans_temp:
                                            final_ans.append(a[0])
                                        if len(final_ans)!=0:
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '59b'
                                            q['question'] = st.replace("< ith x tick>",t)
                                            q['answer'] = final_ans
                                            qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        st = "Which major ticks on independent axis has the value of <Y title> less than that at < ith x tick>, arranged in the increasing order of difference between their <Y title> values?"
                        st = st.replace("<Y title>",'"'+js['task7']['y_label']+'"')
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    for t in texts:
                                        for i in js['task7']['extrapolated'].keys():
                                            for j in js['task7']['extrapolated'][i]:
                                                if float(j[0]) == float(t):
                                                    val1 = j[1]
                                        ans = list()
                                        for i in js['task7']['extrapolated'].keys():
                                            for j in js['task7']['extrapolated'][i]:
                                                if j[1]<val1:
                                                    ans.append((str(j[0]),abs(j[1]-val1)))
                                        ans_temp = sorted(ans,key=lambda x:x[1])
                                        final_ans = list()
                                        for a in ans_temp:
                                            final_ans.append(a[0])
                                        if len(final_ans)!=0:
                                            q = dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '59b'
                                            q['question'] = st.replace("< ith x tick>",t)
                                            q['answer'] = final_ans
                                            qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                        st = "In how many <plural form of X title>, is the value of <Y title> for <legend label> greater than <N> <units>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        if js['task7']['figure_type'] == 'bars' and 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            for i in legend_labels:
                                dat = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'].split() == i.split():
                                        for j1 in j['data']:
                                            if 'y' in j1.keys():
                                                dat.append(j1['y'])
                                            elif 'y2' in j1.keys():
                                                dat.append(j1['y2'])
                                if len(dat)>1:
                                    n = random.uniform(min(dat),max(dat))
                                    qu = st.replace("<N> <units>",str(n))
                                    ans = 0
                                    for j in dat:
                                        if j>n:
                                            ans+=1
                                    q=dict()
                                    q['taxonomy id'] = '4b'
                                    q['QID'] = 60
                                    q['question'] = qu.replace("<legend label>",i)
                                    q['answer'] = ans
                                    qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                        st = "What <plural form of X title> have the value of <Y title> for <legend label> greater than <N> <units>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        if js['task7']['figure_type'] == 'bars' and 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            for i in legend_labels:
                                dat = list()
                                data1 = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'].split() == i.split():
                                        for j1 in j['data']:
                                            if 'y' in j1.keys():
                                                dat.append([j1['x'],j1['y']])
                                                data1.append(j1['y'])
                                            elif 'y2' in j1.keys():
                                                dat.append([j1['x'],j1['y2']])
                                                data1.append(j1['y2'])
                                if len(dat)>1:
                                    n = random.uniform(min(data1),max(data1))
                                    qu = st.replace("<N> <units>",str(n))
                                    ans = list()
                                    for j in dat:
                                        if j[1]>n:
                                            ans.append(j[0])
                                    if len(ans)!=0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '60b'
                                        q['question'] = qu.replace("<legend label>",i)
                                        q['answer'] = tuple(ans)
                                        qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                        st = "What <plural form of X title> have the value of <Y title> for <legend label> greater than <N> <units>, arranged in increasing order of difference between value of <Y title> and <N> <units>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        if js['task7']['figure_type'] == 'bars' and 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            for i in legend_labels:
                                dat = list()
                                data1 = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'].split() == i.split():
                                        for j1 in j['data']:
                                            if 'y' in j1.keys():
                                                dat.append([j1['x'],j1['y']])
                                                data1.append(j1['y'])
                                            elif 'y2' in j1.keys():
                                                dat.append([j1['x'],j1['y2']])
                                                data1.append(j1['y2'])
                                if len(dat)>1:
                                    n = random.uniform(min(data1),max(data1))
                                    qu = st.replace("<N> <units>",str(n))
                                    ans_temp = list()
                                    for j in dat:
                                        if j[1]>n:
                                            ans_temp.append([j[0],abs(j[1]-n)])
                                    ans_temp = sorted(ans_temp,key = lambda x:x[1])
                                    ans = list()
                                    for a in ans_temp:
                                        ans.append(a[0])
                                    if len(ans)!=0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '60d'
                                        q['question'] = qu.replace("<legend label>",i)
                                        q['answer'] = tuple(ans)
                                        qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            legend_labels = list()
                            st = "For how many major ticks on independent axis, is the value of <Y title> for <legend label> greater than <N> <units>?"
                            st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            if 'legends' in js['task7'].keys():
                                legend_labels = js['task7']['legends']
                            elif 'mark_labels_lst' in js['task7'].keys():
                                legend_labels = js['task7']['mark_labels_lst']
                            elif len(js['task7']['name_of_data_series'])>1:
                                legend_labels = js['task7']['name_of_data_series']
                            if len(legend_labels)!=0:
                                for i in legend_labels:
                                    if i in js['task7']['extrapolated'].keys():
                                        dat = list()
                                        for j in js['task7']['extrapolated'].keys():
                                            if j == i:
                                                for j1 in js['task7']['extrapolated'][j]:
                                                    dat.append(j1[1])
                                    if len(dat)>1:
                                        n = random.uniform(min(dat),max(dat))
                                        qu = st.replace("<legend label>",i)
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '60a'
                                        q['question'] = qu.replace("<N> <units>",str(n))
                                        ans = 0
                                        for a in dat:
                                            if a>n:
                                                ans+=1
                                        q['answer'] = ans
                                        qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            legend_labels = list()
                            st = "For what major ticks on independent axis, is the value of <Y title> for <legend label> greater than <N> <units>?"
                            st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            if 'legends' in js['task7'].keys():
                                legend_labels = js['task7']['legends']
                            elif 'mark_labels_lst' in js['task7'].keys():
                                legend_labels = js['task7']['mark_labels_lst']
                            elif len(js['task7']['name_of_data_series'])>1:
                                legend_labels = js['task7']['name_of_data_series']
                            if len(legend_labels)!=0:
                                for i in legend_labels:
                                    if i in js['task7']['extrapolated'].keys():
                                        dat = list()
                                        data1 = list()
                                        for j in js['task7']['extrapolated'].keys():
                                            if j == i:
                                                for j1 in js['task7']['extrapolated'][j]:
                                                    dat.append([j1[0],j1[1]])
                                                    data1.append(j1[1])
                                    if len(dat)>1:
                                        n = random.uniform(min(data1),max(data1))
                                        qu = st.replace("<legend label>",i)
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '60c'
                                        q['question'] = qu.replace("<N> <units>",str(n))
                                        ans = list()
                                        for a in dat:
                                            if a[1]>n:
                                                ans.append(a[0])
                                        if len(ans)!=0:
                                            q['answer'] = tuple(ans)
                                            qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            legend_labels = list()
                            st = "What major ticks on independent axis have the value of <Y title> for <legend label> greater than <N> <units>, arranged in increasing order of difference between value of <Y title> and <N> units?"
                            st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                            if 'legends' in js['task7'].keys():
                                legend_labels = js['task7']['legends']
                            elif 'mark_labels_lst' in js['task7'].keys():
                                legend_labels = js['task7']['mark_labels_lst']
                            elif len(js['task7']['name_of_data_series'])>1:
                                legend_labels = js['task7']['name_of_data_series']
                            if len(legend_labels)!=0:
                                for i in legend_labels:
                                    if i in js['task7']['extrapolated'].keys():
                                        dat = list()
                                        data1 = list()
                                        for j in js['task7']['extrapolated'].keys():
                                            if j == i:
                                                for j1 in js['task7']['extrapolated'][j]:
                                                    dat.append([j1[0],j1[1]])
                                                    data1.append(j1[1])
                                    if len(dat)>1:
                                        n = random.uniform(min(data1),max(data1))
                                        qu = st.replace("<legend label>",i)
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '60e'
                                        q['question'] = qu.replace("<N> <units>",str(n))
                                        ans_temp = list()
                                        for a in dat:
                                            if a[1]>n:
                                                ans_temp.append([a[0],abs(a[1]-n)])
                                        ans_temp = sorted(ans_temp,key=lambda x:x[1])
                                        ans = list()
                                        for a in ans_temp:
                                            ans.append(a[0])
                                        if len(ans)!=0:
                                            q['answer'] = tuple(ans)
                                            qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "What is the ratio of the values of <Y title> for <legend label1> at < ith x tick> to that at <jthx tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            if title_horizontal == "yes":
                                if 'x_ticks_from_top' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_top'])!=0:
                                        legend_label = js['task7']['legends']
                                        texts = js['task7']['x_ticks_from_top']
                                        for i in legend_label:
                                            for j in range(len(texts)-1):
                                                for j1 in range(j+1,len(texts)):
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == i:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == texts[j].split():
                                                                    if 'y' in k1.keys():
                                                                        val1 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val1 = k1['y2']
                                                                elif k1['x'].split() == texts[j1].split():
                                                                    if 'y' in k1.keys():
                                                                        val2 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val2 = k1['y2']
                                                    if val2!=0:
                                                        q=dict()
                                                        q['taxonomy id'] = '4b'
                                                        q['QID'] = 61
                                                        qu = st.replace("<legend label1>",i)
                                                        qu = qu.replace("< ith x tick>",texts[j])
                                                        q['question'] = qu.replace("<jthx tick>",texts[j1])
                                                        q['answer'] = val1/val2
                                                        qa.append(q)
                            else:
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])!=0:
                                        legend_label = js['task7']['legends']
                                        texts = js['task7']['x_ticks_from_left']
                                        for i in legend_label:
                                            for j in range(len(texts)-1):
                                                for j1 in range(j+1,len(texts)):
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == i:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == texts[j].split():
                                                                    if 'y' in k1.keys():
                                                                        val1 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val1 = k1['y2']
                                                                elif k1['x'].split() == texts[j1].split():
                                                                    if 'y' in k1.keys():
                                                                        val2 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val2 = k1['y2']
                                                    if val2!=0:
                                                        q=dict()
                                                        q['taxonomy id'] = '4b'
                                                        q['QID'] = 61
                                                        qu = st.replace("<legend label1>",i)
                                                        qu = qu.replace("< ith x tick>",texts[j])
                                                        q['question'] = qu.replace("<jthx tick>",texts[j1])
                                                        q['answer'] = val1/val2
                                                        qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    legend_label = list()
                                    if 'legends' in js['task7'].keys():
                                        legend_label = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legend_label = js['task7']['mark_labels_lst']
                                    if len(legend_label)!=0:
                                        for i in legend_label:
                                            for j in range(len(texts)-1):
                                                for j1 in range(j+1,len(texts)):
                                                    for k in js['task7']['extrapolated'].keys():
                                                        if k == i:
                                                            for k1 in js['task7']['extrapolated'][k]:
                                                                if float(k1[0]) == float(texts[j]):
                                                                    val1 = k1[1]
                                                                elif float(k1[0]) == float(texts[j1]):
                                                                    val2 = k1[1]
                                                    if val2!=0:
                                                        q=dict()
                                                        q['taxonomy id'] = '4b'
                                                        q['QID'] = 61
                                                        qu = st.replace("<legend label1>",i)
                                                        qu = qu.replace("< ith x tick>",texts[j])
                                                        q['question'] = qu.replace("<jthx tick>",texts[j1])
                                                        q['answer'] = val1/val2
                                                        qa.append(q)
                                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "Is the value of <Y title> for <legend label> at < ith x tick> less than that at < jth x tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            if title_horizontal == "yes":
                                if 'x_ticks_from_top' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_top'])!=0:
                                        legend_label = js['task7']['legends']
                                        texts = js['task7']['x_ticks_from_top']
                                        for i in legend_label:
                                            for j in range(len(texts)-1):
                                                for j1 in range(j+1,len(texts)):
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == i:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == texts[j].split():
                                                                    if 'y' in k1.keys():
                                                                        val1 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val1 = k1['y2']
                                                                elif k1['x'].split() == texts[j1].split():
                                                                    if 'y' in k1.keys():
                                                                        val2 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val2 = k1['y2']
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = 62
                                                    qu = st.replace("<legend label>",i)
                                                    qu = qu.replace("< ith x tick>",texts[j])
                                                    q['question'] = qu.replace("< jth x tick>",texts[j1])
                                                    q['answer'] = 'yes' if val1<val2 else 'no'
                                                    qa.append(q)
                            else:
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])!=0:
                                        legend_label = js['task7']['legends']
                                        texts = js['task7']['x_ticks_from_left']
                                        for i in legend_label:
                                            for j in range(len(texts)-1):
                                                for j1 in range(j+1,len(texts)):
                                                    for k in js['task6']['output']['data series']:
                                                        if k['name'] == i:
                                                            for k1 in k['data']:
                                                                if k1['x'].split() == texts[j].split():
                                                                    if 'y' in k1.keys():
                                                                        val1 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val1 = k1['y2']
                                                                elif k1['x'].split() == texts[j1].split():
                                                                    if 'y' in k1.keys():
                                                                        val2 = k1['y']
                                                                    elif 'y2' in k1.keys():
                                                                        val2 = k1['y2']
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = 62
                                                    qu = st.replace("<legend label>",i)
                                                    qu = qu.replace("< ith x tick>",texts[j])
                                                    q['question'] = qu.replace("< jth x tick>",texts[j1])
                                                    q['answer'] = 'yes' if val1<val2 else 'no'
                                                    qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    legend_label = list()
                                    if 'legends' in js['task7'].keys():
                                        legend_label = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legend_label = js['task7']['mark_labels_lst']
                                    if len(legend_label)!=0:
                                        for i in legend_label:
                                            for j in range(len(texts)-1):
                                                for j1 in range(j+1,len(texts)):
                                                    for k in js['task7']['extrapolated'].keys():
                                                        if k == i:
                                                            for k1 in js['task7']['extrapolated'][k]:
                                                                if float(k1[0]) == float(texts[j]):
                                                                    val1 = k1[1]
                                                                elif float(k1[0]) == float(texts[j1]):
                                                                    val2 = k1[1]
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = 62
                                                    qu = st.replace("<legend label>",i)
                                                    qu = qu.replace("< ith x tick>",texts[j])
                                                    q['question'] = qu.replace("< jth x tick>",texts[j1])
                                                    q['answer'] = 'yes' if val1<val2 else 'no'
                                                    qa.append(q)

            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and 'legends' in js['task7'].keys():
                        st = "Which major ticks on independent axis has the value of <Y title> for <legend label> less than that at <ith x tick>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    legend_labels = js['task7']['legends']
                                    for l in legend_labels:
                                        for t in texts:
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if j['x'].split() == t.split():
                                                            if 'y' in j.keys():
                                                                val1 = j['y']
                                                            elif 'y2' in j.keys():
                                                                val1 = j['y2']
                                            ans = list()
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if 'y' in j.keys():
                                                            if j['y']<val1:
                                                                ans.append(j['x'])
                                                        elif 'y2' in j.keys():
                                                            if j['y2']<val1:
                                                                ans.append(j['x'])
                                            if len(ans)!=0:
                                                q = dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '62a'
                                                qu = st.replace("<legend label>","'"+l+"'")
                                                q['question'] = qu.replace("<ith x tick>",t)
                                                q['answer'] = ans
                                                qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    legend_labels = js['task7']['legends']
                                    for l in legend_labels:
                                        for t in texts:
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if j['x'].split() == t.split():
                                                            if 'y' in j.keys():
                                                                val1 = j['y']
                                                            elif 'y2' in j.keys():
                                                                val1 = j['y2']
                                            ans = list()
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if 'y' in j.keys():
                                                            if j['y']<val1:
                                                                ans.append(j['x'])
                                                        elif 'y2' in j.keys():
                                                            if j['y2']<val1:
                                                                ans.append(j['x'])
                                            if len(ans)!=0:
                                                q = dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '62a'
                                                qu = st.replace("<legend label>","'"+l+"'")
                                                q['question'] = qu.replace("<ith x tick>",t)
                                                q['answer'] = ans
                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        st = "Which major ticks on independent axis has the value of <Y title> for <legend label> less than that at <ith x tick>?"
                        st = st.replace("<Y title>",'"'+js['task7']['y_label']+'"')
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    legend_label = list()
                                    if 'legends' in js['task7'].keys():
                                        legend_label = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legend_label = js['task7']['mark_labels_lst']
                                    if len(legend_label)!=0:
                                        for l in legend_label:
                                            for t in texts:
                                                for i in js['task7']['extrapolated'].keys():
                                                    if i.split() == l.split():
                                                        for j in js['task7']['extrapolated'][i]:
                                                            if float(j[0]) == float(t):
                                                                val1 = j[1]
                                                ans = list()
                                                for i in js['task7']['extrapolated'].keys():
                                                    if i.split() == l.split():
                                                        for j in js['task7']['extrapolated'][i]:
                                                            if j[1]<val1:
                                                                ans.append(str(j[0]))
                                                if len(ans)!=0:
                                                    q = dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = '62a'
                                                    qu = st.replace("<legend label>",l)
                                                    q['question'] = qu.replace("<ith x tick>",t)
                                                    q['answer'] = ans
                                                    qa.append(q)
                                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars' and 'legends' in js['task7'].keys():
                        st = "Which major ticks on independent axis has the value of <Y title> for <legend label> less than that at <ith x tick>, arranged in the increasing order of difference between their <Y title> values?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    legend_labels = js['task7']['legends']
                                    for l in legend_labels:
                                        for t in texts:
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if j['x'].split() == t.split():
                                                            if 'y' in j.keys():
                                                                val1 = j['y']
                                                            elif 'y2' in j.keys():
                                                                val1 = j['y2']
                                            ans = list()
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if 'y' in j.keys():
                                                            if j['y']<val1:
                                                                ans.append([j['x'],abs(j['y']-val1)])
                                                        elif 'y2' in j.keys():
                                                            if j['y2']<val1:
                                                                ans.append([j['x'],abs(j['y2']-val1)])
                                            ans_temp = sorted(ans,key = lambda x:x[1])
                                            final_ans = list()
                                            for a in ans_temp:
                                                final_ans.append(a[0])
                                            if len(final_ans)!=0:
                                                q = dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '62b'
                                                qu = st.replace("<legend label>","'"+l+"'")
                                                q['question'] = qu.replace("<ith x tick>",t)
                                                q['answer'] = final_ans
                                                qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    legend_labels = js['task7']['legends']
                                    for l in legend_labels:
                                        for t in texts:
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if j['x'].split() == t.split():
                                                            if 'y' in j.keys():
                                                                val1 = j['y']
                                                            elif 'y2' in j.keys():
                                                                val1 = j['y2']
                                            ans = list()
                                            for i in js['task6']['output']['data series']:
                                                if i['name'].split() == l.split():
                                                    for j in i['data']:
                                                        if 'y' in j.keys():
                                                            if j['y']<val1:
                                                                ans.append([j['x'],abs(j['y']-val1)])
                                                        elif 'y2' in j.keys():
                                                            if j['y2']<val1:
                                                                ans.append([j['x'],abs(j['y2']-val1)])
                                            ans_temp = sorted(ans,key=lambda x:x[1])
                                            final_ans = list()
                                            for a in ans_temp:
                                                final_ans.append(a[0])
                                            if len(final_ans)!=0:
                                                q = dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '62b'
                                                qu = st.replace("<legend label>","'"+l+"'")
                                                q['question'] = qu.replace("<ith x tick>",t)
                                                q['answer'] = final_ans
                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        st = "Which major ticks on independent axis has the value of <Y title> for <legend label> less than that at <ith x tick>, arranged in the increasing order of difference between their <Y title> values?"
                        st = st.replace("<Y title>",'"'+js['task7']['y_label']+'"')
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    legend_label = list()
                                    if 'legends' in js['task7'].keys():
                                        legend_label = js['task7']['legends']
                                    elif 'mark_labels_lst' in js['task7'].keys():
                                        legend_label = js['task7']['mark_labels_lst']
                                    if len(legend_label)!=0:
                                        for l in legend_label:
                                            for t in texts:
                                                for i in js['task7']['extrapolated'].keys():
                                                    if i.split() == l.split():
                                                        for j in js['task7']['extrapolated'][i]:
                                                            if float(j[0]) == float(t):
                                                                val1 = j[1]
                                                ans = list()
                                                for i in js['task7']['extrapolated'].keys():
                                                    if i.split() == l.split():
                                                        for j in js['task7']['extrapolated'][i]:
                                                            if j[1]<val1:
                                                                ans.append([str(j[0]),abs(j[1]-val1)])
                                                ans_temp = sorted(ans,key=lambda x:x[1])
                                                final_ans = list()
                                                for a in ans_temp:
                                                    final_ans.append(a[0])
                                                if len(final_ans)!=0:
                                                    q = dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = '62b'
                                                    qu = st.replace("<legend label>",l)
                                                    q['question'] = qu.replace("<ith x tick>",t)
                                                    q['answer'] = final_ans
                                                    qa.append(q)
                                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "Is the difference between the value of <Y title> at <ithx tick> and < jthx tick> greater than the difference between any two <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == "yes":
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    dat = list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append(j['y'])
                                            elif 'y2' in j.keys():
                                                dat.append(j['y2'])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append(abs(dat[i]-dat[j]))
                                    for i in range(len(texts)-1):
                                        for j in range(i+1,len(texts)):
                                            for k in js['task6']['output']['data series']:
                                                for k1 in k['data']:
                                                    if k1['x'].split() == texts[i].split():
                                                        if 'y' in k1.keys():
                                                            val1 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val1 = k1['y2']
                                                    elif k1['x'].split() == texts[j].split():
                                                        if 'y' in k1.keys():
                                                            val2 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val2 = k1['y2']
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = 63
                                            qu = st.replace("<ithx tick>",texts[i])
                                            q['question'] = qu.replace("< jthx tick>",texts[j])
                                            ans = any(abs(val1-val2)>x for x in diff)
                                            q['answer'] = 'yes' if ans==True else 'no'
                                            qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat = list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append(j['y'])
                                            elif 'y2' in j.keys():
                                                dat.append(j['y2'])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append(abs(dat[i]-dat[j]))
                                    for i in range(len(texts)-1):
                                        for j in range(i+1,len(texts)):
                                            for k in js['task6']['output']['data series']:
                                                for k1 in k['data']:
                                                    if k1['x'].split() == texts[i].split():
                                                        if 'y' in k1.keys():
                                                            val1 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val1 = k1['y2']
                                                    elif k1['x'].split() == texts[j].split():
                                                        if 'y' in k1.keys():
                                                            val2 = k1['y']
                                                        elif 'y2' in k1.keys():
                                                            val2 = k1['y2']
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = 63
                                            qu = st.replace("<ithx tick>",texts[i])
                                            q['question'] = qu.replace("< jthx tick>",texts[j])
                                            ans = any(abs(val1-val2)>x for x in diff)
                                            q['answer'] = 'yes' if ans==True else 'no'
                                            qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "Is the difference between the value of <Y title> at <ithx tick> and < jthx tick> greater than the difference between any two major ticks on independent axis?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines' or js['task7']['figure_type'] == 'scatter points':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    dat = list()
                                    for i in js['task7']['extrapolated'].keys():
                                        for j in js['task7']['extrapolated'][i]:
                                            dat.append(j[1])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append(abs(dat[i]-dat[j]))
                                    for i in range(len(texts)-1):
                                        for j in range(i+1,len(texts)):
                                            for k in js['task7']['extrapolated'].keys():
                                                for k1 in js['task7']['extrapolated'][k]:
                                                    if float(k1[0]) == float(texts[i]):
                                                        val1 = k1[1]
                                                    elif float(k1[0]) == float(texts[j]):
                                                        val2 = k1[1]
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '63a'
                                            qu = st.replace("<ithx tick>",texts[i])
                                            q['question'] = qu.replace("< jthx tick>",texts[j])
                                            ans = any(abs(val1-val2)>x for x in diff)
                                            q['answer'] = 'yes' if ans==True else 'no'
                                            qa.append(q)
                                            
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "Which pairs of major ticks on independent axis have a greater difference than <ithx tick> and < jthx tick>, for the value of <Y title>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    dat = list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append([j['x'],j['y']])
                                            elif 'y2' in j.keys():
                                                dat.append([j['x'],j['y2']])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append([dat[i][0],dat[j][0],abs(dat[i][1]-dat[j][1])])
                                    for t in range(len(texts)-1):
                                        for t1 in range(t+1,len(texts)):
                                            for i in js['task6']['output']['data series']:
                                                for j in i['data']:
                                                    if j['x'].split() == texts[t].split():
                                                        if 'y' in j.keys():
                                                            val1 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val1 = j['y2']
                                                    elif j['x'].split() == texts[t1].split():
                                                        if 'y' in j.keys():
                                                            val2 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val2 = j['y2']
                                            sub = abs(val1-val2)
                                            ans = list()
                                            for i in diff:
                                                if i[2]>sub:
                                                    ans.append((i[0],i[1]))
                                            if len(ans)!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '63b'
                                                qu = st.replace("<ithx tick>","'"+texts[t]+"'")
                                                q['question'] = qu.replace("< jthx tick>","'"+texts[t1]+"'")
                                                q['answer'] = ans
                                                qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat = list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append([j['x'],j['y']])
                                            elif 'y2' in j.keys():
                                                dat.append([j['x'],j['y2']])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append([dat[i][0],dat[j][0],abs(dat[i][1]-dat[j][1])])
                                    for t in range(len(texts)-1):
                                        for t1 in range(t+1,len(texts)):
                                            for i in js['task6']['output']['data series']:
                                                for j in i['data']:
                                                    if j['x'].split() == texts[t].split():
                                                        if 'y' in j.keys():
                                                            val1 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val1 = j['y2']
                                                    elif j['x'].split() == texts[t1].split():
                                                        if 'y' in j.keys():
                                                            val2 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val2 = j['y2']
                                            sub = abs(val1-val2)
                                            ans = list()
                                            for i in diff:
                                                if i[2]>sub:
                                                    ans.append((i[0],i[1]))
                                            if len(ans)!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '63b'
                                                qu = st.replace("<ithx tick>","'"+texts[t]+"'")
                                                q['question'] = qu.replace("< jthx tick>","'"+texts[t1]+"'")
                                                q['answer'] = ans
                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        st = "Which pairs of major ticks on independent axis have a greater difference than <ithx tick> and < jthx tick>, for the value of <Y title>?"
                        st = st.replace("<Y title>",'"'+js['task7']['y_label']+'"')
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat = list()
                                    for i in js['task7']['extrapolated'].keys():
                                        for j in js['task7']['extrapolated'][i]:
                                            dat.append([j[0],j[1]])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append([dat[i][0],dat[j][0],abs(dat[i][1]-dat[j][1])])
                                    for t in range(len(texts)-1):
                                        for t1 in range(t+1,len(texts)):
                                            for i in js['task7']['extrapolated'].keys():
                                                for j in js['task7']['extrapolated'][i]:
                                                    if float(j[0]) == float(texts[t]):
                                                        val1 = j[1]
                                                    elif float(j[0]) == float(texts[t1]):
                                                        val2 = j[1]
                                            sub = abs(val1-val2)
                                            ans = list()
                                            for i in diff:
                                                if i[2]>sub:
                                                    ans.append((i[0],i[1]))
                                            if len(ans)!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '63b'
                                                qu = st.replace("<ithx tick>",texts[t])
                                                q['question'] = qu.replace("< jthx tick>",texts[t1])
                                                q['answer'] = ans
                                                qa.append(q)
                                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    if js['task7']['figure_type'] == 'bars':
                        st = "Which pairs of major ticks on independent axis have a greater difference than <ithx tick> and < jthx tick>, for the value of <Y title>, arranged in the increasing order of difference?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if title_horizontal == 'yes':
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    dat = list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append([j['x'],j['y']])
                                            elif 'y2' in j.keys():
                                                dat.append([j['x'],j['y2']])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append([dat[i][0],dat[j][0],abs(dat[i][1]-dat[j][1])])
                                    for t in range(len(texts)-1):
                                        for t1 in range(t+1,len(texts)):
                                            for i in js['task6']['output']['data series']:
                                                for j in i['data']:
                                                    if j['x'].split() == texts[t].split():
                                                        if 'y' in j.keys():
                                                            val1 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val1 = j['y2']
                                                    elif j['x'].split() == texts[t1].split():
                                                        if 'y' in j.keys():
                                                            val2 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val2 = j['y2']
                                            sub = abs(val1-val2)
                                            ans = list()
                                            for i in diff:
                                                if i[2]>sub:
                                                    ans.append((i[0],i[1],i[2]))
                                            ans_temp = sorted(ans,key=lambda x:x[2])
                                            final_ans = list()
                                            for a in ans_temp:
                                                final_ans.append((a[0],a[1]))
                                            if len(final_ans)!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '63c'
                                                qu = st.replace("<ithx tick>","'"+texts[t]+"'")
                                                q['question'] = qu.replace("< jthx tick>","'"+texts[t1]+"'")
                                                q['answer'] = final_ans
                                                qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat = list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append([j['x'],j['y']])
                                            elif 'y2' in j.keys():
                                                dat.append([j['x'],j['y2']])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append([dat[i][0],dat[j][0],abs(dat[i][1]-dat[j][1])])
                                    for t in range(len(texts)-1):
                                        for t1 in range(t+1,len(texts)):
                                            for i in js['task6']['output']['data series']:
                                                for j in i['data']:
                                                    if j['x'].split() == texts[t].split():
                                                        if 'y' in j.keys():
                                                            val1 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val1 = j['y2']
                                                    elif j['x'].split() == texts[t1].split():
                                                        if 'y' in j.keys():
                                                            val2 = j['y']
                                                        elif 'y2' in j.keys():
                                                            val2 = j['y2']
                                            sub = abs(val1-val2)
                                            ans = list()
                                            for i in diff:
                                                if i[2]>sub:
                                                    ans.append((i[0],i[1],i[2]))
                                            ans_temp = sorted(ans,key=lambda x:x[2])
                                            final_ans = list()
                                            for a in ans_temp:
                                                final_ans.append((a[0],a[1]))
                                            if len(final_ans)!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '63c'
                                                qu = st.replace("<ithx tick>","'"+texts[t]+"'")
                                                q['question'] = qu.replace("< jthx tick>","'"+texts[t1]+"'")
                                                q['answer'] = final_ans
                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        st = "Which pairs of major ticks on independent axis have a greater difference than <ithx tick> and < jthx tick>, for the value of <Y title>, arranged in the increasing order of difference?"
                        st = st.replace("<Y title>",'"'+js['task7']['y_label']+'"')
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat = list()
                                    for i in js['task7']['extrapolated'].keys():
                                        for j in js['task7']['extrapolated'][i]:
                                            dat.append([j[0],j[1]])
                                    diff = list()
                                    for i in range(len(dat)-1):
                                        for j in range(i+1,len(dat)):
                                            diff.append([dat[i][0],dat[j][0],abs(dat[i][1]-dat[j][1])])
                                    for t in range(len(texts)-1):
                                        for t1 in range(t+1,len(texts)):
                                            for i in js['task7']['extrapolated'].keys():
                                                for j in js['task7']['extrapolated'][i]:
                                                    if float(j[0]) == float(texts[t]):
                                                        val1 = j[1]
                                                    elif float(j[0]) == float(texts[t1]):
                                                        val2 = j[1]
                                            sub = abs(val1-val2)
                                            ans = list()
                                            for i in diff:
                                                if i[2]>sub:
                                                    ans.append((i[0],i[1],i[2]))
                                            ans_temp = sorted(ans,key=lambda x:x[2])
                                            final_ans = list()
                                            for a in ans_temp:
                                                final_ans.append((a[0],a[1]))
                                            if len(final_ans)!=0:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '63c'
                                                qu = st.replace("<ithx tick>",texts[t])
                                                q['question'] = qu.replace("< jthx tick>",texts[t1])
                                                q['answer'] = final_ans
                                                qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "What is the difference between the maximum and the second maximum value of <Y title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        dat = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat.append(j['y2'])
                        if len(dat)>1:
                            max_val = max(dat)
                            dat.remove(max_val)
                            answer = abs(max_val - max(dat))
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 64
                            q['question'] = st
                            q['answer'] = answer
                            qa.append(q)
                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "Is the sum of the value of <Y title> in < ithx tick> and <(i + 1)thx tick> greater than the maximum value of <Y title> across all <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == "yes":
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    dat,question,answer = list(),list(),list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append(j['y'])
                                            elif 'y2' in j.keys():
                                                dat.append(j['y2'])
                                    max_val = max(dat)
                                    for i in range(len(texts)-1):
                                        ques = st.replace("< ithx tick>",texts[i])
                                        question.append(ques.replace("<(i + 1)thx tick>",texts[i+1]))
                                        for j in js['task6']['output']['data series']:
                                            for k in j['data']:
                                                if k['x'] == texts[i]:
                                                    if 'y' in k.keys():
                                                        val1 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val1 = k['y2']
                                                elif k['x'] == texts[i+1]:
                                                    if 'y' in k.keys():
                                                        val2 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val2 = k['y2']
                                        answer.append("yes" if (val1+val2)>max_val else "no")
                                    for i in range(len(question)):
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 65
                                        q['question'] = question[i]
                                        q['answer'] = answer[i]
                                        qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat,question,answer = list(),list(),list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append(j['y'])
                                            elif 'y2' in j.keys():
                                                dat.append(j['y2'])
                                    max_val = max(dat)
                                    for i in range(len(texts)-1):
                                        ques = st.replace("< ithx tick>",texts[i])
                                        question.append(ques.replace("<(i + 1)thx tick>",texts[i+1]))
                                        for j in js['task6']['output']['data series']:
                                            for k in j['data']:
                                                if k['x'] == texts[i]:
                                                    if 'y' in k.keys():
                                                        val1 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val1 = k['y2']
                                                elif k['x'] == texts[i+1]:
                                                    if 'y' in k.keys():
                                                        val2 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val2 = k['y2']
                                        answer.append("yes" if (val1+val2)>max_val else "no")
                                    for i in range(len(question)):
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 65
                                        q['question'] = question[i]
                                        q['answer'] = answer[i]
                                        qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "Is the sum of the value of <Y title> in < ithx tick> and <(i + 1)thx tick> greater than the maximum value of <Y title> across all major ticks on independent axis?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    dat,question,answer = list(),list(),list()
                                    for i in js['task7']['extrapolated'].keys():
                                        for j in js['task7']['extrapolated'][i]:
                                            dat.append(j[1])
                                    max_val = max(dat)
                                    for i in range(len(texts)-1):
                                        ques = st.replace("< ithx tick>",texts[i])
                                        question.append(ques.replace("<(i + 1)thx tick>",texts[i+1]))
                                        for j in js['task7']['extrapolated'].keys():
                                            for k in js['task7']['extrapolated'][j]:
                                                if float(k[0]) == float(texts[i]):
                                                    val1 = k[1]
                                                elif float(k[0]) == float(texts[i+1]):
                                                    val2 = k[1]
                                        answer.append("yes" if (val1+val2)>max_val else "no")
                                    for i in range(len(question)):
                                        if len(answer)!=0:
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '65a'
                                            q['question'] = question[i]
                                            q['answer'] = answer[i]
                                            qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "For what pair of consecutive major ticks on independent axis, is the sum of the value of <Y title>, greater than the maximum value of <Y title> across all <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if title_horizontal == "yes":
                            if 'x_ticks_from_top' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_top'])!=0:
                                    texts = js['task7']['x_ticks_from_top']
                                    dat,answer = list(),list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append(j['y'])
                                            elif 'y2' in j.keys():
                                                dat.append(j['y2'])
                                    max_val = max(dat)
                                    for i in range(len(texts)-1):
                                        for j in js['task6']['output']['data series']:
                                            for k in j['data']:
                                                if k['x'] == texts[i]:
                                                    if 'y' in k.keys():
                                                        val1 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val1 = k['y2']
                                                elif k['x'] == texts[i+1]:
                                                    if 'y' in k.keys():
                                                        val2 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val2 = k['y2']
                                        if (val1+val2)>max_val:
                                            answer.append((texts[i],texts[i+1]))
                                    if len(answer)!=0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '65b'
                                        q['question'] = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                                        q['answer'] = tuple(answer)
                                        qa.append(q)
                        else:
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    texts = js['task7']['x_ticks_from_left']
                                    dat,answer = list(),list()
                                    for i in js['task6']['output']['data series']:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                dat.append(j['y'])
                                            elif 'y2' in j.keys():
                                                dat.append(j['y2'])
                                    max_val = max(dat)
                                    for i in range(len(texts)-1):
                                        for j in js['task6']['output']['data series']:
                                            for k in j['data']:
                                                if k['x'] == texts[i]:
                                                    if 'y' in k.keys():
                                                        val1 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val1 = k['y2']
                                                elif k['x'] == texts[i+1]:
                                                    if 'y' in k.keys():
                                                        val2 = k['y']
                                                    elif 'y2' in k.keys():
                                                        val2 = k['y2']
                                        if (val1+val2)>max_val:
                                            answer.append((texts[i],texts[i+1]))
                                    if len(answer)!=0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '65b'
                                        q['question'] = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                                        q['answer'] = tuple(answer)
                                        qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "For what pair of consecutive major ticks on independent axis, is the sum of the value of <Y title> greater than the maximum value of <Y title> across all major ticks on independent axis?"
                    if js['task7']['figure_type'] == 'lines':
                        if 'x_ticks_from_left' in js['task7'].keys():
                            if len(js['task7']['x_ticks_from_left'])!=0:
                                if 'extrapolated' in js['task7'].keys():
                                    texts = js['task7']['x_ticks_from_left']
                                    dat,answer = list(),list()
                                    for i in js['task7']['extrapolated'].keys():
                                        for j in js['task7']['extrapolated'][i]:
                                            dat.append(j[1])
                                    max_val = max(dat)
                                    for i in range(len(texts)-1):
                                        for j in js['task7']['extrapolated'].keys():
                                            for k in js['task7']['extrapolated'][j]:
                                                if float(k[0]) == float(texts[i]):
                                                    val1 = k[1]
                                                elif float(k[0]) == float(texts[i+1]):
                                                    val2 = k[1]
                                        if (val1+val2)>max_val:
                                            answer.append((texts[i],texts[i+1]))
                                    if len(answer)!=0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '65c'
                                        q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                                        q['answer'] = tuple(answer)
                                        qa.append(q)
                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "What is the difference between the maximum and the minimum value of <Y title>?"
                    if js['task7']['figure_type'] == 'bars':
                        dat = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat.append(j['y2'])
                        max_val = max(dat)
                        min_val = min(dat)
                        answer = abs(max_val - min_val)
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 66
                        q['question'] = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        q['answer'] = answer
                        qa.append(q)
                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "In how many <plural form of X title>, is the value of <Y title> greater than the average value of <Y title> taken over all <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        dat = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat.append(j['y2'])
                        aver = np.mean(dat)
                        count = 0
                        for i in dat:
                            if i>aver:
                                count+=1
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 67
                        q['question'] = st
                        q['answer'] = count
                        qa.append(q)
                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'mark_labels_lst' not in js['task7'].keys() and 'legends' not in js['task7'].keys() and len(js['task7']['name_of_data_series'])==1:
                    st = "What is the count of major ticks on independent axis, where the value of <Y title> greater than the average value of <Y title> taken over all major ticks on independent axis?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            dat = list()
                            for i in js['task7']['extrapolated'].keys():
                                for j in js['task7']['extrapolated'][i]:
                                    dat.append(j[1])
                            aver = np.mean(dat)
                            count = 0
                            for i in dat:
                                if i>aver:
                                    count+=1
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = '67a'
                            q['question'] = st
                            q['answer'] = count
                            qa.append(q)
                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>1:
                    st = "Is the difference between the value of <Y title> for <legend label1> at < ithx tick> and at < jthx tick> greater than the difference between the value of <Y title> for <legend label2> at < ithx tick> and at < jthx tick>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            if len(js['task7']['legends'])>1:
                                if title_horizontal == "yes":
                                    if 'x_ticks_from_top' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_top'])!=0:
                                            legends = js['task7']['legends']
                                            ticks = js['task7']['x_ticks_from_top']
                                            for i in range(len(legends)-1):
                                                for j in range(i+1,len(legends)):
                                                    for i1 in range(len(ticks)-1):
                                                        for j1 in range(i1+1,len(ticks)):
                                                            for k in js['task6']['output']['data series']:
                                                                if legends[i].split() == k['name'].split():
                                                                    for k1 in k['data']:
                                                                        if ticks[i1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val1 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val1 = k1['y2']
                                                                        elif ticks[j1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val2 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val2 = k1['y2']
                                                                elif legends[j].split() == k['name'].split():
                                                                    for k1 in k['data']:
                                                                        if ticks[i1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val3 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val3 = k1['y2']
                                                                        elif ticks[j1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val4 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val4 = k1['y2']
                                                            q=dict()
                                                            q['taxonomy id'] = '4b'
                                                            q['QID'] = 68
                                                            qu = st.replace("<legend label1>",legends[i])
                                                            qu = qu.replace("<legend label2>",legends[j])
                                                            qu = qu.replace("< ithx tick>",ticks[i1])
                                                            q['question'] = qu.replace("< jthx tick>",ticks[j1])
                                                            q['answer'] = 'yes' if abs(val1-val2)>abs(val3-val4) else 'no'
                                                            qa.append(q)
                                else:
                                    if 'x_ticks_from_left' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_left'])!=0:
                                            legends = js['task7']['legends']
                                            ticks = js['task7']['x_ticks_from_left']
                                            for i in range(len(legends)-1):
                                                for j in range(i+1,len(legends)):
                                                    for i1 in range(len(ticks)-1):
                                                        for j1 in range(i1+1,len(ticks)):
                                                            for k in js['task6']['output']['data series']:
                                                                if legends[i].split() == k['name'].split():
                                                                    for k1 in k['data']:
                                                                        if ticks[i1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val1 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val1 = k1['y2']
                                                                        elif ticks[j1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val2 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val2 = k1['y2']
                                                                elif legends[j].split() == k['name'].split():
                                                                    for k1 in k['data']:
                                                                        if ticks[i1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val3 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val3 = k1['y2']
                                                                        elif ticks[j1].split() == k1['x'].split():
                                                                            if 'y' in k1.keys():
                                                                                val4 = k1['y']
                                                                            elif 'y2' in k1.keys():
                                                                                val4 = k1['y2']
                                                            q=dict()
                                                            q['taxonomy id'] = '4b'
                                                            q['QID'] = 68
                                                            qu = st.replace("<legend label1>",legends[i])
                                                            qu = qu.replace("<legend label2>",legends[j])
                                                            qu = qu.replace("< ithx tick>",ticks[i1])
                                                            q['question'] = qu.replace("< jthx tick>",ticks[j1])
                                                            q['answer'] = 'yes' if abs(val1-val2)>abs(val3-val4) else 'no'
                                                            qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    if 'legends' in js['task7'].keys() or 'mark_labels_lst' in js['task7'].keys() or len(js['task7']['name_of_data_series'])>1:
                                        ticks = js['task7']['x_ticks_from_left']
                                        if 'legends' in js['task7'].keys():
                                            legends = js['task7']['legends']
                                        elif 'mark_labels_lst' in js['task7'].keys():
                                            legends = js['task7']['mark_labels_lst']
                                        elif len(js['task7']['name_of_data_series'])>1:
                                            legends = js['task7']['name_of_data_series']
                                        if len(legends)>1:
                                            for i in range(len(legends)-1):
                                                for j in range(i+1,len(legends)):
                                                    for i1 in range(len(ticks)-1):
                                                        for j1 in range(i1+1,len(ticks)):
                                                            for k in js['task7']['extrapolated'].keys():
                                                                if legends[i].split() == k.split():
                                                                    for k1 in js['task7']['extrapolated'][k]:
                                                                        if float(ticks[i1]) == float(k1[0]):
                                                                            val1 = k1[1]
                                                                        elif float(ticks[j1]) == float(k1[0]):
                                                                            val2 = k1[1]
                                                                elif legends[j].split() == k.split():
                                                                    for k1 in js['task7']['extrapolated'][k]:
                                                                        if float(ticks[i1]) == float(k1[0]):
                                                                            val3 = k1[1]
                                                                        elif float(ticks[j1]) == float(k1[0]):
                                                                            val4 = k1[1]
                                                            q=dict()
                                                            q['taxonomy id'] = '4b'
                                                            q['QID'] = 68
                                                            qu = st.replace("<legend label1>",legends[i])
                                                            qu = qu.replace("<legend label2>",legends[j])
                                                            qu = qu.replace("< ithx tick>",ticks[i1])
                                                            q['question'] = qu.replace("< jthx tick>",ticks[j1])
                                                            q['answer'] = 'yes' if abs(val1-val2)>abs(val3-val4) else 'no'
                                                            qa.append(q)
                                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>1:
                    st = "Which pairs of major ticks on independent axis for <legend label1> have a greater difference than at <ithx tick> and < jthx tick> for <legend label2>, for the value of <Y title>?"
                    st = st.replace("<Y title>",js['task7']['y_label'])
                    if js['task7']['figure_type'] == 'bars':
                        if 'legends' in js['task7'].keys():
                            if len(js['task7']['legends'])>1:
                                if title_horizontal == "yes":
                                    if 'x_ticks_from_top' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_top'])!=0:
                                            legends = js['task7']['legends']
                                            ticks = js['task7']['x_ticks_from_top']
                                            for l1 in range(len(legends)-1):
                                                diff_l1 = list()
                                                temp_lst = list()
                                                for i in js['task6']['output']['data series']:
                                                    if i['name'].split() == legends[l1].split():
                                                        for j in i['data']:
                                                            if 'y' in j.keys():
                                                                temp_lst.append([j['x'],j['y']])
                                                            elif 'y2' in j.keys():
                                                                temp_lst.append([j['x'],j['y2']])
                                                for i in range(len(temp_lst)-1):
                                                    for j in range(i+1,len(temp_lst)):
                                                        diff_l1.append([temp_lst[i][0],temp_lst[j][0],abs(temp_lst[i][1]-temp_lst[j][1])])
                                                for l2 in range(l1+1,len(legends)):
                                                    for t1 in range(len(ticks)-1):
                                                        for t2 in range(t1+1,len(ticks)):
                                                            for i in js['task6']['output']['data series']:
                                                                if i['name'].split() == legends[l2].split():
                                                                    for j in i['data']:
                                                                        if ticks[t1].split() == j['x'].split():
                                                                            if 'y' in j.keys():
                                                                                val1 = j['y']
                                                                            elif 'y2' in j.keys():
                                                                                val1 = j['y2']
                                                                        elif ticks[t2].split() == j['x'].split():
                                                                            if 'y' in j.keys():
                                                                                val2 = j['y']
                                                                            elif 'y2' in j.keys():
                                                                                val2 = j['y2']
                                                            ans = list()
                                                            for i in diff_l1:
                                                                if i[2]>abs(val1 - val2):
                                                                    ans.append((i[0],i[1]))
                                                            if len(ans)!=0:
                                                                q=dict()
                                                                q['taxonomy id'] = '4b'
                                                                q['QID'] = '68a'
                                                                qu = st.replace("<legend label1>",'"'+legends[l1]+'"')
                                                                qu = qu.replace("<legend label2>",'"'+legends[l2]+'"')
                                                                qu = qu.replace("<ithx tick>",'"'+ticks[t1]+'"')
                                                                q['question'] = qu.replace("< jthx tick>",'"'+ticks[t2]+'"')
                                                                q['answer'] = ans
                                                                qa.append(q)
                                else:
                                    if 'x_ticks_from_left' in js['task7'].keys():
                                        if len(js['task7']['x_ticks_from_left'])!=0:
                                            legends = js['task7']['legends']
                                            ticks = js['task7']['x_ticks_from_left']
                                            for l1 in range(len(legends)-1):
                                                diff_l1 = list()
                                                temp_lst = list()
                                                for i in js['task6']['output']['data series']:
                                                    if i['name'].split() == legends[l1].split():
                                                        for j in i['data']:
                                                            if 'y' in j.keys():
                                                                temp_lst.append([j['x'],j['y']])
                                                            elif 'y2' in j.keys():
                                                                temp_lst.append([j['x'],j['y2']])
                                                for i in range(len(temp_lst)-1):
                                                    for j in range(i+1,len(temp_lst)):
                                                        diff_l1.append([temp_lst[i][0],temp_lst[j][0],abs(temp_lst[i][1]-temp_lst[j][1])])
                                                for l2 in range(l1+1,len(legends)):
                                                    for t1 in range(len(ticks)-1):
                                                        for t2 in range(t1+1,len(ticks)):
                                                            for i in js['task6']['output']['data series']:
                                                                if i['name'].split() == legends[l2].split():
                                                                    for j in i['data']:
                                                                        if ticks[t1].split() == j['x'].split():
                                                                            if 'y' in j.keys():
                                                                                val1 = j['y']
                                                                            elif 'y2' in j.keys():
                                                                                val1 = j['y2']
                                                                        elif ticks[t2].split() == j['x'].split():
                                                                            if 'y' in j.keys():
                                                                                val2 = j['y']
                                                                            elif 'y2' in j.keys():
                                                                                val2 = j['y2']
                                                            ans = list()
                                                            for i in diff_l1:
                                                                if i[2]>abs(val1 - val2):
                                                                    ans.append((i[0],i[1]))
                                                            if len(ans)!=0:
                                                                q=dict()
                                                                q['taxonomy id'] = '4b'
                                                                q['QID'] = '68a'
                                                                qu = st.replace("<legend label1>",'"'+legends[l1]+'"')
                                                                qu = qu.replace("<legend label2>",'"'+legends[l2]+'"')
                                                                qu = qu.replace("<ithx tick>",'"'+ticks[t1]+'"')
                                                                q['question'] = qu.replace("< jthx tick>",'"'+ticks[t2]+'"')
                                                                q['answer'] = ans
                                                                qa.append(q)
                    elif js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            if 'x_ticks_from_left' in js['task7'].keys():
                                if len(js['task7']['x_ticks_from_left'])!=0:
                                    if 'legends' in js['task7'].keys() or 'mark_labels_lst' in js['task7'].keys() or len(js['task7']['name_of_data_series'])>1:
                                        ticks = js['task7']['x_ticks_from_left']
                                        if 'legends' in js['task7'].keys():
                                            legends = js['task7']['legends']
                                        elif 'mark_labels_lst' in js['task7'].keys():
                                            legends = js['task7']['mark_labels_lst']
                                        elif len(js['task7']['name_of_data_series'])>1:
                                            legends = js['task7']['name_of_data_series']
                                        if len(legends)>1:
                                            for l1 in range(len(legends)-1):
                                                diff_l1 = list()
                                                temp_lst = list()
                                                for i in js['task7']['extrapolated'].keys():
                                                    if i.split() == legends[l1].split():
                                                        for j in js['task7']['extrapolated'][i]:
                                                            temp_lst.append([j[0],j[1]])
                                                for i in range(len(temp_lst)-1):
                                                    for j in range(i+1,len(temp_lst)):
                                                        diff_l1.append([temp_lst[i][0],temp_lst[j][0],abs(float(temp_lst[i][1])-float(temp_lst[j][1]))])
                                                for l2 in range(l1+1,len(legends)):
                                                    for t1 in range(len(ticks)-1):
                                                        for t2 in range(t1+1,len(ticks)):
                                                            for i in js['task7']['extrapolated'].keys():
                                                                if i.split() == legends[l2].split():
                                                                    for j in js['task7']['extrapolated'][i]:
                                                                        if float(ticks[t1]) == float(j[0]):
                                                                            val1 = j[1]
                                                                        elif float(ticks[t2]) == float(j[0]):
                                                                            val2 = j[1]
                                                            ans = list()
                                                            for i in diff_l1:
                                                                if i[2]>abs(val1 - val2):
                                                                    ans.append((i[0],i[1]))
                                                            if len(ans)!=0:
                                                                q=dict()
                                                                q['taxonomy id'] = '4b'
                                                                q['QID'] = '68a'
                                                                qu = st.replace("<legend label1>",'"'+legends[l1]+'"')
                                                                qu = qu.replace("<legend label2>",'"'+legends[l2]+'"')
                                                                qu = qu.replace("<ithx tick>",'"'+ticks[t1]+'"')
                                                                q['question'] = qu.replace("< jthx tick>",'"'+ticks[t2]+'"')
                                                                q['answer'] = ans
                                                                qa.append(q)
                                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "What is the difference between the values of maximum and the second maximum <Y title> for <legend label>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars' and len(js['task7']['name_of_data_series'])>0:
                        if 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            questions,answers = list(),list()
                            for i in legend_labels:
                                questions.append(st.replace("<legend label>",i))
                                dat = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                if len(dat)!=0:
                                    max_val = max(dat)
                                    if len(dat)>=2:
                                        dat.remove(max_val)
                                        answers.append(abs(max_val-max(dat)))
                                    else:
                                        answers.append(None)
                            if len(answers)!=0 and len(questions)==len(answers):
                                for i in range(len(questions)):
                                    if answers[i]!=None:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 69
                                        q['question'] = questions[i]
                                        q['answer'] = answers[i]
                                        qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    st = "What is the difference between the maximum and minimum value of <Y title> for <legend label>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'bars' and len(js['task7']['name_of_data_series'])>0:
                        if 'legends' in js['task7'].keys():
                            legend_labels = js['task7']['legends']
                            questions,answers = list(),list()
                            for i in legend_labels:
                                questions.append(st.replace("<legend label>",i))
                                dat = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                if len(dat)!=0:
                                    if len(dat)>=2:
                                        answers.append(abs(max(dat)-min(dat)))
                                    else:
                                        answers.append(None)
                            if len(answers)!=0 and len(questions)==len(answers):
                                for i in range(len(questions)):
                                    if answers[i]!=None:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 70
                                        q['question'] = questions[i]
                                        q['answer'] = answers[i]
                                        qa.append(q)
                                            
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>0:
                    st = "In how many <plural form of X title>, is the value of <Y title> for <legend label> greater than the average value of <Y title> for <legend label> taken over all <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        legend_labels = js['task7']['legends']
                        questions,answers = list(),list()
                        for i in legend_labels:
                            questions.append(st.replace("<legend label>",i))
                            dat = list()
                            for j in js['task6']['output']['data series']:
                                if j['name'] == i:
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            dat.append(k['y'])
                                        elif 'y2' in k.keys():
                                            dat.append(k['y2'])
                            if len(dat)!=0:
                                aver = np.mean(dat)
                                count = 0
                                for j in dat:
                                    if j>aver:
                                        count+=1
                                answers.append(count)
                            else:
                                answers.append(count)
                        if len(answers)!=0:
                            for i in range(len(questions)):
                                q = dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 71
                                q['question'] = questions[i]
                                q['answer'] = answers[i]
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>0:
                    st = "What is the count of major ticks on independent axis where the value of <Y title> for <legend label> is greater than the average value of <Y title> for <legend label> taken over all major ticks on independent axis?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            legend_labels = list()
                            if 'legends' in js['task7'].keys():
                                legend_labels = js['task7']['legends']
                            elif 'mark_labels_lst' in js['task7'].keys():
                                legend_labels = js['task7']['mark_labels_lst']
                            if len(legend_labels)!=0:
                                questions,answers = list(),list()
                                for i in legend_labels:
                                    questions.append(st.replace("<legend label>",i))
                                    dat = list()
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                    if len(dat)!=0:
                                        aver = np.mean(dat)
                                        count = 0
                                        for j in dat:
                                            if j>aver:
                                                count+=1
                                        answers.append(count)
                                    else:
                                        answers.append(count)
                                if len(answers)!=0:
                                    for i in range(len(questions)):
                                        q = dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '71a'
                                        q['question'] = questions[i]
                                        q['answer'] = answers[i]
                                        qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>0:
                    st = "What <plural form of X title> have the value of <Y title> for <legend label> greater than the average value of <Y title> for <legend label> taken over all <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        legend_labels = js['task7']['legends']
                        questions,answers = list(),list()
                        for i in legend_labels:
                            qu = st.replace("<legend label>",i)
                            dat = list()
                            for j in js['task6']['output']['data series']:
                                if j['name'] == i:
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            dat.append(k['y'])
                                        elif 'y2' in k.keys():
                                            dat.append(k['y2'])
                            if len(dat)!=0:
                                aver = np.mean(dat)
                                answers = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                if k['y']>aver:
                                                    answers.append(k['x'])
                                            elif 'y2' in k.keys():
                                                if k['y2']>aver:
                                                    answers.append(k['x'])
                                if len(answers)!=0:
                                    q=dict()
                                    q['taxonomy id'] = '4b'
                                    q['QID'] = '71b'
                                    q['question'] = qu
                                    q['answer'] = tuple(answers)
                                    qa.append(q)
                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>0:
                    st = "What <plural form of X title> have the value of <Y title> for <legend label> greater than the average value of <Y title> for <legend label> taken over all <plural form of X title>, arranged in increasing order of difference between the value of <Y title> for <legend label> and average value of <Y title> for <legend label> taken over all <plural form of X title>?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                    if js['task7']['figure_type'] == 'bars':
                        legend_labels = js['task7']['legends']
                        questions,answers = list(),list()
                        for i in legend_labels:
                            qu = st.replace("<legend label>",i)
                            dat = list()
                            for j in js['task6']['output']['data series']:
                                if j['name'] == i:
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            dat.append(k['y'])
                                        elif 'y2' in k.keys():
                                            dat.append(k['y2'])
                            if len(dat)!=0:
                                aver = np.mean(dat)
                                answers_temp = list()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                if k['y']>aver:
                                                    answers_temp.append([k['x'],abs(k['y']-aver)])
                                            elif 'y2' in k.keys():
                                                if k['y2']>aver:
                                                    answers_temp.append([k['x'],abs(k['y2']-aver)])
                                answers_temp = sorted(answers_temp,key = lambda x:x[1])
                                answers = list()
                                for j in answers_temp:
                                    answers.append(j[0])
                                if len(answers)!=0:
                                    q=dict()
                                    q['taxonomy id'] = '4b'
                                    q['QID'] = '71d'
                                    q['question'] = qu
                                    q['answer'] = tuple(answers)
                                    qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and len(js['task7']['name_of_data_series'])>0:
                    st = "What major ticks on independent axis have the value of <Y title> for <legend label> greater than the average value of <Y title> for <legend label> taken over all major ticks on independent axis?"
                    st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                    if js['task7']['figure_type'] == 'lines':
                        if 'extrapolated' in js['task7'].keys():
                            legend_labels = list()
                            if 'legends' in js['task7'].keys():
                                legend_labels = js['task7']['legends']
                            elif 'mark_labels_lst' in js['task7'].keys():
                                legend_labels = js['task7']['mark_labels_lst']
                            if len(legend_labels)!=0:
                                for i in legend_labels:
                                    qu = st.replace("<legend label>",i)
                                    dat = list()
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                    if len(dat)!=0:
                                        aver = np.mean(dat)
                                        answers = list()
                                        for j in js['task7']['extrapolated'].keys():
                                            if j == i:
                                                for k in js['task7']['extrapolated'][j]:
                                                    if k[1]>aver:
                                                        answers.append(k[0])
                                        if len(answers)!=0:
                                            q=dict()
                                            q['taxonomy id'] = '4b'
                                            q['QID'] = '71c'
                                            q['question'] = qu
                                            q['answer'] = tuple(answers)
                                            qa.append(q)
                                        
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' in js['task7'].keys():
                    if len(js['task7']['legends'])>2:
                        st = "Is it the case that in every <singular form of X title>, the sum of the value of <Y title> for <legend label1> and <legend label2> is greater than the value of  <Y title> for <legend label3>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        st = st.replace("<singular form of X title>","'"+js['task7']['x_label']+"'")
                        if js['task7']['figure_type'] == 'bars' and len(js['task7']['name_of_data_series'])>2:
                            legend_labels = js['task7']['legends']
                            questions,answers,legend_stack = list(),list(),list()
                            for i in range(len(legend_labels)-2):
                                for j in range(i+1,len(legend_labels)-1):
                                    for k in range(j+1,len(legend_labels)):
                                        ques = st.replace("<legend label1>",legend_labels[i])
                                        ques = ques.replace("<legend label2>",legend_labels[j])
                                        questions.append(ques.replace("<legend label3>",legend_labels[k]))
                                        legend_stack.append([legend_labels[i],legend_labels[j],legend_labels[k]])
                            for q in range(len(questions)):
                                leg1,leg2,leg3 = legend_stack[q][0],legend_stack[q][1],legend_stack[q][2]
                                s_dict,leg3_lst = dict(),dict()
                                for i in js['task6']['output']['data series']:
                                    for j in i['data']:
                                        s_dict[j['x']]=0
                                        leg3_lst[j['x']]=0
                                for i in js['task6']['output']['data series']:
                                    if i['name']==leg1 or i['name']==leg2:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                s_dict[j['x']]+=j['y']
                                            elif 'y2' in j.keys():
                                                s_dict[j['x']]+=j['y2']
                                    elif i['name']==leg3:
                                        for j in i['data']:
                                            if 'y' in j.keys():
                                                leg3_lst[j['x']]+=j['y']
                                            elif 'y2' in j.keys():
                                                leg3_lst[j['x']]+=j['y2']
                                f=1
                                for i in s_dict:
                                    if s_dict[i]<leg3_lst[i]:
                                        f=0
                                answers.append("yes" if f==1 else "no")
                            for i in range(len(questions)):
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 72
                                q['question'] = questions[i]
                                q['answer'] = answers[i]
                                qa.append(q)
                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' in js['task7'].keys():
                    if len(js['task7']['legends'])>1:
                        st = "Is the sum of the values of <Y title> for <legend label1> at < ithx tick> and < jthx tick> greater than the maximum value of <Y title> for <legend label2> across all <plural form of X title>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            legend_labels = js['task7']['legends']
                            if title_horizontal == "yes":
                                if 'x_ticks_from_top' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_top'])>1:
                                        ticks_label = js['task7']['x_ticks_from_top']
                                        legends = js['task7']['legends']
                                        for i in range(len(legends)):
                                            for k in range(len(legends)):
                                                if legends[i].split() != legends[k].split():
                                                    for j in range(len(ticks_label)-1):
                                                        for k1 in range(j+1,len(ticks_label)):
                                                            dat = list()
                                                            for i1 in js['task6']['output']['data series']:
                                                                if i1['name'].split() == legends[i].split():
                                                                    for j1 in i1['data']:
                                                                        if j1['x'].split() == ticks_label[j].split():
                                                                            if 'y' in j1.keys():
                                                                                val1 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val1 = j1['y2']
                                                                        elif j1['x'].split() == ticks_label[k1].split():
                                                                            if 'y' in j1.keys():
                                                                                val2 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val2 = j1['y2']
                                                                elif i1['name'].split() == legends[k].split():
                                                                    for j1 in i1['data']:
                                                                        if 'y' in j1.keys():
                                                                            dat.append(j1['y'])
                                                                        elif 'y2' in j1.keys():
                                                                            dat.append(j1['y2'])
                                                            q=dict()
                                                            q['taxonomy id'] = '4b'
                                                            q['QID'] = 73
                                                            if len(dat)>1:
                                                                qu = st.replace("<legend label1>",legends[i])
                                                                qu = qu.replace("<legend label2>",legends[k])
                                                                qu = qu.replace("< ithx tick>",ticks_label[j])
                                                                q['question'] = qu.replace("< jthx tick>",ticks_label[k1])
                                                                q['answer'] = 'yes' if abs(val1+val2)>max(dat) else 'no'
                                                                qa.append(q)
                            else:
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])>1:
                                        ticks_label = js['task7']['x_ticks_from_left']
                                        legends = js['task7']['legends']
                                        for i in range(len(legends)):
                                            for k in range(len(legends)):
                                                if legends[i].split() != legends[k].split():
                                                    for j in range(len(ticks_label)-1):
                                                        for k1 in range(j+1,len(ticks_label)):
                                                            dat = list()
                                                            for i1 in js['task6']['output']['data series']:
                                                                if i1['name'].split() == legends[i].split():
                                                                    for j1 in i1['data']:
                                                                        if j1['x'].split() == ticks_label[j].split():
                                                                            if 'y' in j1.keys():
                                                                                val1 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val1 = j1['y2']
                                                                        elif j1['x'].split() == ticks_label[k1].split():
                                                                            if 'y' in j1.keys():
                                                                                val2 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val2 = j1['y2']
                                                                elif i1['name'].split() == legends[k].split():
                                                                    for j1 in i1['data']:
                                                                        if 'y' in j1.keys():
                                                                            dat.append(j1['y'])
                                                                        elif 'y2' in j1.keys():
                                                                            dat.append(j1['y2'])
                                                            q=dict()
                                                            q['taxonomy id'] = '4b'
                                                            q['QID'] = 73
                                                            if len(dat)>1:
                                                                qu = st.replace("<legend label1>",legends[i])
                                                                qu = qu.replace("<legend label2>",legends[k])
                                                                qu = qu.replace("< ithx tick>",ticks_label[j])
                                                                q['question'] = qu.replace("< jthx tick>",ticks_label[k1])
                                                                q['answer'] = 'yes' if abs(val1+val2)>max(dat) else 'no'
                                                                qa.append(q)
                                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    legend_labels = list()
                    if 'legends' in js['task7'].keys():
                        legend_labels = js['task7']['legends']
                    if 'mark_labels_lst' in js['task7'].keys():
                        legend_labels = js['task7']['mark_labels_lst']
                    if len(legend_labels)>1:
                        st = "Is the sum of the values of <Y title> for <legend label1> at < ithx tick> and < jthx tick> greater than the maximum value of <Y title> for <legend label2> across all major ticks on independent axis?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'lines':
                            if 'extrapolated' in js['task7'].keys():
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])!=0:
                                        ticks_label = js['task7']['x_ticks_from_left']
                                        questions,stack_pos = list(),list()
                                        for i in range(len(legend_labels)):
                                            for j in range(len(legend_labels)):
                                                for k in js['task7']['extrapolated'].keys():
                                                    for t1 in range(len(ticks_label)-1):
                                                        for t2 in range(t1+1,len(ticks_label)):
                                                            q_temp = st.replace("<legend label1>",legend_labels[i])
                                                            q_temp = q_temp.replace("<legend label2>",legend_labels[j])
                                                            q_temp = q_temp.replace("< ithx tick>",ticks_label[t1])
                                                            questions.append(q_temp.replace("< jthx tick>",ticks_label[t2]))
                                                            stack_pos.append([legend_labels[i],ticks_label[t1],ticks_label[t2],legend_labels[j]])
                                        answers = list()
                                        for q in range(len(questions)):
                                            s=0
                                            max_lst = list()
                                            leg1,t1,t2,leg2 = stack_pos[q][0],stack_pos[q][1],stack_pos[q][2],stack_pos[q][3]
                                            for i in js['task7']['extrapolated'].keys():
                                                if i==leg1 and i!=leg2:
                                                    for j in js['task7']['extrapolated'][i]:
                                                        if float(j[0])==float(t1):
                                                            s+=j[1]
                                                        elif float(j[0])==float(t2):
                                                            s+=j[1]
                                                elif i==leg2 and i!=leg1:
                                                    for j in js['task7']['extrapolated'][i]:
                                                        max_lst.append(j[1])
                                                elif i==leg1 and i==leg2:
                                                    for j in js['task7']['extrapolated'][i]:
                                                        if float(j[0])==float(t1):
                                                            s+=j[1]
                                                        elif float(j[0])==float(t2):
                                                            s+=j[1]
                                                        max_lst.append(j[1])
                                            if len(max_lst)!=0:
                                                ans = "yes" if s>max(max_lst) else "no"
                                            else:
                                                ans = None
                                            answers.append(ans)
                                        for i in range(len(questions)):
                                            if answers[i]!=None:
                                                q=dict()
                                                q['taxonomy id'] = '4b'
                                                q['QID'] = '73a'
                                                q['question'] = questions[i]
                                                q['answer'] = answers[i]
                                                qa.append(q)
                                                
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys() and 'legends' in js['task7'].keys():
                    if len(js['task7']['legends'])>1:
                        st = "For what pair of major ticks on independent axis, is the sum of the values of <Y title> for <legend label1>, greater than the maximum value of <Y title> for <legend label2> across all <plural form of X title>?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        st = st.replace("<plural form of X title>","'"+js['task7']['x_label']+"'")
                        if js['task7']['figure_type'] == 'bars':
                            legend_labels = js['task7']['legends']
                            if title_horizontal == "yes":
                                if 'x_ticks_from_top' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_top'])>1:
                                        ticks_label = js['task7']['x_ticks_from_top']
                                        legends = js['task7']['legends']
                                        for i in range(len(legends)):
                                            for k in range(len(legends)):
                                                answers = list()
                                                if legends[i].split() != legends[k].split():
                                                    for j in range(len(ticks_label)-1):
                                                        for k1 in range(j+1,len(ticks_label)):
                                                            dat = list()
                                                            for i1 in js['task6']['output']['data series']:
                                                                if i1['name'].split() == legends[i].split():
                                                                    for j1 in i1['data']:
                                                                        if j1['x'].split() == ticks_label[j].split():
                                                                            if 'y' in j1.keys():
                                                                                val1 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val1 = j1['y2']
                                                                        elif j1['x'].split() == ticks_label[k1].split():
                                                                            if 'y' in j1.keys():
                                                                                val2 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val2 = j1['y2']
                                                                elif i1['name'].split() == legends[k].split():
                                                                    for j1 in i1['data']:
                                                                        if 'y' in j1.keys():
                                                                            dat.append(j1['y'])
                                                                        elif 'y2' in j1.keys():
                                                                            dat.append(j1['y2'])
                                                            if len(dat)!=0:
                                                                if abs(val1+val2)>max(dat):
                                                                    answers.append((ticks_label[j],ticks_label[k1]))
                                                if len(answers)!=0:
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = '73b'
                                                    qu = st.replace("<legend label1>",legends[i])
                                                    q['question'] = qu.replace("<legend label2>",legends[k])
                                                    q['answer'] = answers
                                                    qa.append(q)
                            else:
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])>1:
                                        ticks_label = js['task7']['x_ticks_from_left']
                                        legends = js['task7']['legends']
                                        for i in range(len(legends)):
                                            for k in range(len(legends)):
                                                answers = list()
                                                if legends[i].split() != legends[k].split():
                                                    for j in range(len(ticks_label)-1):
                                                        for k1 in range(j+1,len(ticks_label)):
                                                            dat = list()
                                                            for i1 in js['task6']['output']['data series']:
                                                                if i1['name'].split() == legends[i].split():
                                                                    for j1 in i1['data']:
                                                                        if j1['x'].split() == ticks_label[j].split():
                                                                            if 'y' in j1.keys():
                                                                                val1 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val1 = j1['y2']
                                                                        elif j1['x'].split() == ticks_label[k1].split():
                                                                            if 'y' in j1.keys():
                                                                                val2 = j1['y']
                                                                            elif 'y2' in j1.keys():
                                                                                val2 = j1['y2']
                                                                elif i1['name'].split() == legends[k].split():
                                                                    for j1 in i1['data']:
                                                                        if 'y' in j1.keys():
                                                                            dat.append(j1['y'])
                                                                        elif 'y2' in j1.keys():
                                                                            dat.append(j1['y2'])
                                                            if len(dat)!=0:
                                                                if abs(val1+val2)>max(dat):
                                                                    answers.append((ticks_label[j],ticks_label[k1]))
                                                if len(answers)!=0:
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = '73b'
                                                    qu = st.replace("<legend label1>",legends[i])
                                                    q['question'] = qu.replace("<legend label2>",legends[k])
                                                    q['answer'] = answers
                                                    qa.append(q)
                                                    
            if 'task6' in js.keys() and 'task7' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys() and 'y_label' in js['task7'].keys():
                    if 'legends' in js['task7'].keys():
                        legends = js['task7']['legends']
                    if 'mark_labels_lst' in js['task7'].keys():
                        legends = js['task7']['mark_labels_lst']
                    if len(legend_labels)>1:
                        st = "For what pair of major ticks on independent axis, is the sum of the values of <Y title> for <legend label1>, greater than the maximum value of <Y title> for <legend label2> across all major ticks on independent axis?"
                        st = st.replace("<Y title>","'"+js['task7']['y_label']+"'")
                        if js['task7']['figure_type'] == 'lines':
                            if 'extrapolated' in js['task7'].keys():
                                if 'x_ticks_from_left' in js['task7'].keys():
                                    if len(js['task7']['x_ticks_from_left'])!=0:
                                        ticks_label = js['task7']['x_ticks_from_left']
                                        for i in range(len(legends)):
                                            for k in range(len(legends)):
                                                answers = list()
                                                if legends[i].split() != legends[k].split():
                                                    for j in range(len(ticks_label)-1):
                                                        for k1 in range(j+1,len(ticks_label)):
                                                            dat = list()
                                                            for i1 in js['task7']['extrapolated'].keys():
                                                                if i1.split() == legends[i].split():
                                                                    for j1 in js['task7']['extrapolated'][i1]:
                                                                        if float(j1[0]) == float(ticks_label[j]):
                                                                            val1 = j1[1]
                                                                        elif float(j1[0]) == float(ticks_label[k1]):
                                                                            val2 = j1[1]
                                                                elif i1.split() == legends[k].split():
                                                                    for j1 in js['task7']['extrapolated'][i1]:
                                                                        dat.append(j1[1])
                                                            if len(dat)!=0:
                                                                if abs(val1+val2)>max(dat):
                                                                    answers.append((ticks_label[j],ticks_label[k1]))
                                                if len(answers)!=0:
                                                    q=dict()
                                                    q['taxonomy id'] = '4b'
                                                    q['QID'] = '73c'
                                                    qu = st.replace("<legend label1>",legends[i])
                                                    q['question'] = qu.replace("<legend label2>",legends[k])
                                                    q['answer'] = answers
                                                    qa.append(q)
                                                
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'is_grouped' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        q=dict()
                        q['taxonomy id'] = '1b'
                        q['QID'] = 74
                        q['question'] = 'Are the bars grouped?'
                        q['answer'] = 'yes' if js['task7']['is_grouped'] else 'no'
                        qa.append(q)
                        
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        q=dict()
                        q['taxonomy id'] = '1b'
                        q['QID'] = 75
                        q['question'] = 'Are the bars stacked?'
                        q['answer'] = 'yes' if js['task7']['is_stacked'] else 'no'
                        qa.append(q)
                        
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                    if js['task7']['is_stacked']:
                        if js['task7']['figure_type'] == 'bars':
                            q=dict()
                            q['taxonomy id'] = '1b'
                            q['QID'] = 76
                            q['question'] = 'What is the maximum number of components stacked in every bar?'
                            if 'stacks_per_tick_from_left' in js['task7'].keys():
                                q['answer'] = max(js['task7']['stacks_per_tick_from_left'])
                                qa.append(q)
                            if 'stacks_per_tick_from_top' in js['task7'].keys():
                                q['answer'] = max(js['task7']['stacks_per_tick_from_top'])
                                qa.append(q)
                            
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys() and 'is_stacked' in js['task7'].keys():
                    if js['task7']['is_stacked']:
                        if js['task7']['figure_type'] == 'bars':
                            q=dict()
                            q['taxonomy id'] = '1b'
                            q['QID'] = 77
                            q['question'] = 'What is the minimum number of components stacked in every bar?'
                            if 'stacks_per_tick_from_left' in js['task7'].keys():
                                q['answer'] = min(js['task7']['stacks_per_tick_from_left'])
                                qa.append(q)
                            if 'stacks_per_tick_from_top' in js['task7'].keys():
                                q['answer'] = min(js['task7']['stacks_per_tick_from_top'])
                                qa.append(q)
                                
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '1b'
                        q['QID'] = 78
                        q['question'] = 'How many lines are there in the chart?'
                        if 'number_of_figures' in js['task7'].keys():
                            q['answer'] = js['task7']['number_of_figures']
                            qa.append(q)
                            
            if 'task7' in js.keys():
                st = "Is independent axis categorical or numerical?"
                ticks = list()
                if 'x_ticks_from_left' in js['task7'].keys(): 
                    ticks = js['task7']['x_ticks_from_left']
                elif 'x_ticks_from_top' in js['task7'].keys():
                    ticks = js['task7']['x_ticks_from_top']
                if len(ticks)!=0:
                    q=dict()
                    q['taxonomy id'] = '2a'
                    q['QID'] = 79
                    q['question'] = st
                    f=1
                    for i in ticks:
                        try:
                            ti = float(i)
                        except:
                            f=0
                            break
                    q['answer'] = 'numerical' if f==1 else 'categorical'
                    qa.append(q)
                    
            if 'task7' in js.keys():
                st = "What is the maximum value on independent axis?"
                ticks = list()
                if 'x_ticks_from_left' in js['task7'].keys(): 
                    ticks = js['task7']['x_ticks_from_left']
                elif 'x_ticks_from_top' in js['task7'].keys():
                    ticks = js['task7']['x_ticks_from_top']
                if len(ticks)!=0:
                    q=dict()
                    q['taxonomy id'] = '2a'
                    q['QID'] = 80
                    q['question'] = st
                    f=1
                    tic_x = list()
                    for i in ticks:
                        try:
                            tic_x.append(float(i))
                        except:
                            f=0
                            break
                    if f==1:
                        q['answer'] = max(tic_x)
                        qa.append(q)
                        
            if 'task7' in js.keys():
                st = "What is the starting value on independent axis?"
                ticks = list()
                if 'x_ticks_from_left' in js['task7'].keys(): 
                    ticks = js['task7']['x_ticks_from_left']
                elif 'x_ticks_from_top' in js['task7'].keys():
                    ticks = js['task7']['x_ticks_from_top']
                if len(ticks)!=0:
                    q=dict()
                    q['taxonomy id'] = '2a'
                    q['QID'] = 81
                    q['question'] = st
                    f=1
                    tic_x = list()
                    for i in ticks:
                        try:
                            tic_x.append(float(i))
                        except:
                            f=0
                            break
                    if f==1:
                        q['answer'] = min(tic_x)
                        qa.append(q)
                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None:
                    st = "What is the maximum value on dependent axis?"
                    ticks = list()
                    if 'y_ticks_from_left' in js['task7'].keys(): 
                        ticks = js['task7']['y_ticks_from_left']
                    elif 'y_ticks_from_top' in js['task7'].keys():
                        ticks = js['task7']['y_ticks_from_top']
                    if len(ticks)!=0:
                        q=dict()
                        q['taxonomy id'] = '2c'
                        q['QID'] = 84
                        q['question'] = st
                        f=1
                        tic_x = list()
                        for i in ticks:
                            try:
                                tic_x.append(float(i))
                            except:
                                f=0
                                break
                        dat_list = list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat_list.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat_list.append(j['y2'])
                        if f==1:
                            if max(dat_list)>max(tic_x):
                                q['answer'] = round(max(dat_list))
                                qa.append(q)
                            else:
                                q['answer'] = abs(max(tic_x) - min(tic_x))
                                qa.append(q)
                        
            if 'task7' in js.keys():
                st = "What is the starting value on dependent axis?"
                ticks = list()
                if 'y_ticks_from_left' in js['task7'].keys(): 
                    ticks = js['task7']['y_ticks_from_left']
                elif 'y_ticks_from_top' in js['task7'].keys():
                    ticks = js['task7']['y_ticks_from_top']
                if len(ticks)!=0:
                    q=dict()
                    q['taxonomy id'] = '2c'
                    q['QID'] = 86
                    q['question'] = st
                    f=1
                    tic_x = list()
                    for i in ticks:
                        try:
                            tic_x.append(float(i))
                        except:
                            f=0
                            break
                    if f==1:
                        q['answer'] = min(tic_x)
                        qa.append(q)
                        
            if 'task7' in js.keys() and 'task3' in js.keys():
                if js['task3']!=None:
                    q=dict()
                    q['taxonomy id'] = '2e'
                    q['QID'] = 89
                    q['question'] = 'Is there a legend title?'
                    f=1
                    for i in js['task3']['output']['text_roles']:
                        if i['role'] == 'legend_title':
                            f=0
                    q['answer'] = 'yes' if f==0 else 'no'
                    qa.append(q)
                
            if 'task7' in js.keys() and 'task3' in js.keys() and 'task2' in js.keys():
                if js['task3']!=None and js['task2']!=None:
                    q=dict()
                    q['taxonomy id'] = '2e'
                    q['QID'] = 90
                    q['question'] = 'What is the title of legend?'
                    f=1
                    for i in js['task3']['output']['text_roles']:
                        if i['role'] == 'legend_title':
                            f=0
                            iden = i['id']
                    if f==0:
                        for i in js['task2']['output']['text_blocks']:
                            if i['id'] == iden:
                                tit = i['text']
                        q['answer'] = tit
                        qa.append(q)
                            
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.mean(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 94
                                q['question'] = "Which data series has the highest mean?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.mean(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 95
                                q['question'] = "Which data series has the lowest mean?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.median(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 96
                                q['question'] = "Which data series has the highest median?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.median(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 97
                                q['question'] = "Which data series has the lowest median?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = stats_mode.mode(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 98
                                q['question'] = "Which data series has the highest mode?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = stats_mode.mode(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 99
                                q['question'] = "Which data series has the lowest mode?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.var(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 100
                                q['question'] = "Which data series has the highest variance?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.var(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 101
                                q['question'] = "Which data series has the lowest variance?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.std(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 102
                                q['question'] = "Which data series has the highest standard deviation?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'bars':
                        leg_lab_values = dict()
                        if len(js['task7']['legends'])>1:
                            for i in js['task7']['legends']:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        leg_lab_values[i] = np.std(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 103
                                q['question'] = "Which data series has the lowest standard deviation?"
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 106
                        q['question'] = "Which data series has the highest mean?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.mean(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.mean(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 107
                        q['question'] = "Which data series has the lowest mean?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.mean(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.mean(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 108
                        q['question'] = "Which data series has the highest median?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.median(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.median(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 109
                        q['question'] = "Which data series has the lowest median?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.median(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.median(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 112
                        q['question'] = "Which data series has the highest variance?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.var(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.var(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 113
                        q['question'] = "Which data series has the lowest variance?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.var(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.var(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 114
                        q['question'] = "Which data series has the highest standard deviation?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.std(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.std(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1],reverse = True)
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    legend_label = list()
                    if 'legends' in js['task7'].keys():
                        legend_label = js['task7']['legends']
                    elif 'mark_labels_lst' in js['task7'].keys():
                        legend_label = js['task7']['mark_labels_lst']
                    elif len(js['task7']['name_of_data_series'])>1:
                        legend_label = js['task7']['name_of_data_series']
                    if js['task7']['figure_type'] == 'lines':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 115
                        q['question'] = "Which data series has the lowest standard deviation?"
                        leg_lab_values = dict()
                        if len(legend_label)>1:
                            for i in legend_label:
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        dat = list()
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                dat.append(k['y'])
                                            elif 'y2' in k.keys():
                                                dat.append(k['y2'])
                                        if len(dat)!=0:
                                            leg_lab_values[i] = np.std(dat)
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            dat = list()
                                            for k in js['task7']['extrapolated'][j]:
                                                dat.append(k[1])
                                            if len(dat)!=0:
                                                leg_lab_values[i] = np.std(dat)
                            leg_dict = sorted(leg_lab_values.items(),key = lambda x:x[1])
                            if len(leg_dict)!=0:
                                answers = list()
                                for a in range(len(leg_dict)):
                                    if leg_dict[a][1] == leg_dict[0][1]:
                                        answers.append(leg_dict[a][0])
                                q['answer'] = ",".join(answers)
                                qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    if js['task7']['figure_type']=='lines':
                        legends_lab = list()
                        if 'legends' in js['task7'].keys():
                            legends_lab = js['task7']['legends']
                        elif 'mark_labels_lst' in js['task7'].keys():
                            legends_lab = js['task7']['mark_labels_lst']
                        elif len(js['task7']['name_of_data_series'])>1:
                            legends_lab = js['task7']['name_of_data_series']
                        if len(legends_lab)!=0:
                            st = 'Is the <legend-label> monotonically increasing?'
                            for i in legends_lab:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 116
                                q['question'] = st.replace("<legend-label>",i)
                                lag_dat = dict()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                lag_dat[k['x']] = k['y']
                                            elif 'y2' in k.keys():
                                                lag_dat[k['x']] = k['y2']
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                lag_dat[k[0]] = k[1]
                                if len(lag_dat)!=0:
                                    lag_dat = sorted(lag_dat.items(),key = lambda x:x[0])
                                    flag=1
                                    for j in range(len(lag_dat)-1):
                                        if lag_dat[j][1]>lag_dat[j+1][1]:
                                            flag = 0
                                            break
                                    q['answer'] = 'yes' if flag == 1 else 'no'
                                    qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None:
                    if js['task7']['figure_type']=='lines':
                        legends_lab = list()
                        if 'legends' in js['task7'].keys():
                            legends_lab = js['task7']['legends']
                        elif 'mark_labels_lst' in js['task7'].keys():
                            legends_lab = js['task7']['mark_labels_lst']
                        elif len(js['task7']['name_of_data_series'])>1:
                            legends_lab = js['task7']['name_of_data_series']
                        if len(legends_lab)!=0:
                            st = 'Is the <legend_label> monotonically decreasing?'
                            for i in legends_lab:
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 117
                                q['question'] = st.replace("<legend_label>",i)
                                lag_dat = dict()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                lag_dat[k['x']] = k['y']
                                            elif 'y2' in k.keys():
                                                lag_dat[k['x']] = k['y2']
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                lag_dat[k[0]] = k[1]
                                if len(lag_dat)!=0:
                                    lag_dat = sorted(lag_dat.items(),key = lambda x:x[0])
                                    flag=1
                                    for j in range(len(lag_dat)-1):
                                        if lag_dat[j][1]<lag_dat[j+1][1]:
                                            flag = 0
                                            break
                                    q['answer'] = 'yes' if flag == 1 else 'no'
                                    qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='lines':
                        legends_lab = list()
                        if 'legends' in js['task7'].keys():
                            legends_lab = js['task7']['legends']
                        elif 'mark_labels_lst' in js['task7'].keys():
                            legends_lab = js['task7']['mark_labels_lst']
                        elif len(js['task7']['name_of_data_series'])>1:
                            legends_lab = js['task7']['name_of_data_series']
                        if len(legends_lab)!=0:
                            st = 'Which data series has the minimum difference in <Y_label>?'
                            leg_diff = dict()
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 119
                            q['question'] = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                            for i in legends_lab:
                                lag_dat = dict()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                lag_dat[k['x']] = k['y']
                                            elif 'y2' in k.keys():
                                                lag_dat[k['x']] = k['y2']
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                lag_dat[k[0]] = k[1]
                                if len(lag_dat)!=0:
                                    lag_da = sorted(lag_dat.items(),key = lambda x:x[1])
                                    leg_diff[i] = abs(lag_da[len(lag_da)-1][1] - lag_da[0][1])
                            diff_list = sorted(leg_diff.items(),key=lambda x:x[1])
                            if len(diff_list)!=0:
                                answ_lst = list()
                                for a in diff_list:
                                    if a[1] == diff_list[0][1]:
                                        answ_lst.append(a[0])
                                q['answer'] = ",".join(answ_lst)
                                qa.append(q)
                            
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='lines':
                        legends_lab = list()
                        if 'legends' in js['task7'].keys():
                            legends_lab = js['task7']['legends']
                        elif 'mark_labels_lst' in js['task7'].keys():
                            legends_lab = js['task7']['mark_labels_lst']
                        elif len(js['task7']['name_of_data_series'])>1:
                            legends_lab = js['task7']['name_of_data_series']
                        if len(legends_lab)!=0:
                            st = 'Which data series has the maximum difference in <Y_label>?'
                            leg_diff = dict()
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 120
                            q['question'] = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                            for i in legends_lab:
                                lag_dat = dict()
                                for j in js['task6']['output']['data series']:
                                    if j['name'] == i:
                                        for k in j['data']:
                                            if 'y' in k.keys():
                                                lag_dat[k['x']] = k['y']
                                            elif 'y2' in k.keys():
                                                lag_dat[k['x']] = k['y2']
                                if 'extrapolated' in js['task7'].keys():
                                    for j in js['task7']['extrapolated'].keys():
                                        if j == i:
                                            for k in js['task7']['extrapolated'][j]:
                                                lag_dat[k[0]] = k[1]
                                if len(lag_dat)!=0:
                                    lag_da = sorted(lag_dat.items(),key = lambda x:x[1])
                                    leg_diff[i] = abs(lag_da[len(lag_da)-1][1] - lag_da[0][1])
                            diff_list = sorted(leg_diff.items(),key=lambda x:x[1],reverse = True)
                            if len(diff_list)!=0:
                                answ_lst = list()
                                for a in diff_list:
                                    if a[1] == diff_list[0][1]:
                                        answ_lst.append(a[0])
                                q['answer'] = ",".join(answ_lst)
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys():
                    if js['task7']['figure_type']=='scatter points':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 125
                        q['question'] = "What is the slope of line of best fit?"
                        dat_x,dat_y = list(),list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat_y.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat_y.append(j['y2'])
                                try:
                                    dat_x.append(float(j['x']))
                                except:
                                    break
                        if len(dat_x)!=0:
                            if len(dat_y)!=0:
                                dat_x = np.array(dat_x)
                                dat_y = np.array(dat_y)
                                q['answer'],_ = np.polyfit(dat_x,dat_y,1)
                                qa.append(q)

            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' not in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='scatter points':
                        q=dict()
                        q['taxonomy id'] = '4b'
                        q['QID'] = 129
                        st = "What is the correlation between <X_label> and <Y_label>?"
                        st = st.replace("<X_label>","'"+js['task7']['x_label']+"'")
                        q['question'] = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                        dat_x,dat_y = list(),list()
                        for i in js['task6']['output']['data series']:
                            for j in i['data']:
                                if 'y' in j.keys():
                                    dat_y.append(j['y'])
                                elif 'y2' in j.keys():
                                    dat_y.append(j['y2'])
                                try:
                                    dat_x.append(float(j['x']))
                                except:
                                    break
                        if len(dat_x)>1:
                            if len(dat_y)>1:
                                dat_x = np.array(dat_x)
                                dat_y = np.array(dat_y)
                                q['answer'] = stats_mode.pearsonr(dat_x,dat_y)[0]
                                qa.append(q)
                                
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type']=='scatter points':
                        legends_lab = js['task7']['legends']
                        for i in legends_lab:
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 121
                            st = "Does <legend_label> have low positive correlation?"
                            q['question'] = st.replace("<legend_label>",i)
                            for j in js['task6']['output']['data series']:
                                if i == j['name']:
                                    val_lst_x,val_lst_y = list(),list()
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            val_lst_y.append(k['y'])
                                        elif 'y2' in k.keys():
                                            val_lst_y.append(k['y2'])
                                        try:
                                            val_lst_x.append(float(k['x']))
                                        except:
                                            pass
                                    if len(val_lst_x)>=2 and len(val_lst_y)>=2:
                                        val_lst_x = np.array(val_lst_x)
                                        val_lst_y = np.array(val_lst_y)
                                        correlate = stats_mode.pearsonr(val_lst_x,val_lst_y)[0]
                                        if correlate>0 and correlate<=0.5:
                                            q['answer'] = 'yes'
                                            qa.append(q)
                                        else:
                                            q['answer'] = 'no'
                                            qa.append(q)
                                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type']=='scatter points':
                        legends_lab = js['task7']['legends']
                        for i in legends_lab:
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 122
                            st = "Does <legend_label> have high positive correlation?"
                            q['question'] = st.replace("<legend_label>",i)
                            for j in js['task6']['output']['data series']:
                                if i == j['name']:
                                    val_lst_x,val_lst_y = list(),list()
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            val_lst_y.append(k['y'])
                                        elif 'y2' in k.keys():
                                            val_lst_y.append(k['y2'])
                                        try:
                                            val_lst_x.append(float(k['x']))
                                        except:
                                            pass
                                    if len(val_lst_x)>=2 and len(val_lst_y)>=2:
                                        val_lst_x = np.array(val_lst_x)
                                        val_lst_y = np.array(val_lst_y)
                                        correlate = stats_mode.pearsonr(val_lst_x,val_lst_y)[0]
                                        if correlate<=1 and correlate>0.5:
                                            q['answer'] = 'yes'
                                            qa.append(q)
                                        else:
                                            q['answer'] = 'no'
                                            qa.append(q)
                                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type']=='scatter points':
                        legends_lab = js['task7']['legends']
                        for i in legends_lab:
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 123
                            st = "Does <legend_label> have low negative correlation?"
                            q['question'] = st.replace("<legend_label>",i)
                            for j in js['task6']['output']['data series']:
                                if i == j['name']:
                                    val_lst_x,val_lst_y = list(),list()
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            val_lst_y.append(k['y'])
                                        elif 'y2' in k.keys():
                                            val_lst_y.append(k['y2'])
                                        try:
                                            val_lst_x.append(float(k['x']))
                                        except:
                                            pass
                                    if len(val_lst_x)>=2 and len(val_lst_y)>=2:
                                        val_lst_x = np.array(val_lst_x)
                                        val_lst_y = np.array(val_lst_y)
                                        correlate = stats_mode.pearsonr(val_lst_x,val_lst_y)[0]
                                        if correlate>=-0.5 and correlate<=0:
                                            q['answer'] = 'yes'
                                            qa.append(q)
                                        else:
                                            q['answer'] = 'no'
                                            qa.append(q)
                                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys():
                    if js['task7']['figure_type']=='scatter points':
                        legends_lab = js['task7']['legends']
                        for i in legends_lab:
                            q=dict()
                            q['taxonomy id'] = '4b'
                            q['QID'] = 124
                            st = "Does <legend_label> have high negative correlation?"
                            q['question'] = st.replace("<legend_label>",i)
                            for j in js['task6']['output']['data series']:
                                if i == j['name']:
                                    val_lst_x,val_lst_y = list(),list()
                                    for k in j['data']:
                                        if 'y' in k.keys():
                                            val_lst_y.append(k['y'])
                                        elif 'y2' in k.keys():
                                            val_lst_y.append(k['y2'])
                                        try:
                                            val_lst_x.append(float(k['x']))
                                        except:
                                            pass
                                    if len(val_lst_x)>=2 and len(val_lst_y)>=2:
                                        val_lst_x = np.array(val_lst_x)
                                        val_lst_y = np.array(val_lst_y)
                                        correlate = stats_mode.pearsonr(val_lst_x,val_lst_y)[0]
                                        if correlate<-0.5 and correlate>=-1:
                                            q['answer'] = 'yes'
                                            qa.append(q)
                                        else:
                                            q['answer'] = 'no'
                                            qa.append(q)
                                        
            if 'task3' in js.keys():
                if js['task3']!=None:
                    q=dict()
                    q['taxonomy id'] = '2i'
                    q['QID'] = 91
                    q['question'] = "Other than the graph objects, is there any other text present on the graph plot area?"
                    flag = 1
                    lst_flag = ['legend_label','axis_title','tick_label','chart_title','legend_title']
                    for i in js['task3']['output']['text_roles']:
                        if i['role'] not in lst_flag:
                            flag = 0
                    q['answer'] = 'yes' if flag==0 else 'no'
                    qa.append(q)
                    
            if 'task3' in js.keys() and 'task2' in js.keys():
                if js['task3']!=None and js['task2']!=None:
                    flag = 1
                    iden = list()
                    lst_flag = ['legend_label','axis_title','tick_label','chart_title','legend_title']
                    for i in js['task3']['output']['text_roles']:
                        if i['role'] not in lst_flag:
                            flag = 0
                            iden.append(i['id'])
                    if flag==0:
                        ans = list()
                        for i in js['task2']['output']['text_blocks']:
                            if i['id'] in iden:
                                ans.append(i['text'])
                        q=dict()
                        q['taxonomy id'] = '2i'
                        q['QID'] = '91a'
                        q['question'] = "Other than the graph objects, what other text are present on the graph plot area?"
                        q['answer'] = ans
                        qa.append(q)
                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='bars':
                        legends_lab = js['task7']['legends']
                        if len(legends_lab)>1:
                            st = "For which <plural form of X_label> are the <Y_label> for <legend-label1> greater than <Y_label> for <legend-label2>?"
                            st = st.replace("<plural form of X_label>","'"+js['task7']['x_label']+"'")
                            st = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                            for i in range(len(legends_lab)-1):
                                for j in range(i+1,len(legends_lab)):
                                    leg1,leg2 = list(),list()
                                    for k1 in js['task6']['output']['data series']:
                                        if k1['name'].split() == legends_lab[i].split():
                                            for l1 in k1['data']:
                                                if 'y' in l1.keys():
                                                    leg1.append([l1['x'],l1['y']])
                                                elif 'y2' in l1.keys():
                                                    leg1.append([l1['x'],l1['y2']])
                                        elif k1['name'].split() == legends_lab[j].split():
                                            for l2 in k1['data']:
                                                if 'y' in l2.keys():
                                                    leg2.append([l2['x'],l2['y']])
                                                elif 'y2' in l2.keys():
                                                    leg2.append([l2['x'],l2['y2']])
                                    final_dict = dict()
                                    for a in range(len(leg1)):
                                        for b in range(len(leg2)):
                                            if leg1[a][0].split() == leg2[b][0].split():
                                                final_dict[leg1[a][0]] = [leg1[a][1],leg2[b][1]]
                                    ans = list()
                                    for a in final_dict:
                                        if final_dict[a][0]>final_dict[a][1]:
                                            ans.append(a)
                                    qu = st.replace("<legend-label1>",legends_lab[i])
                                    if len(ans)>0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = str(104)
                                        q['question'] = qu.replace("<legend-label2>",legends_lab[j])
                                        q['answer'] = tuple(ans)
                                        qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='bars':
                        legends_lab = js['task7']['legends']
                        if len(legends_lab)>1:
                            st = "Which <plural form of X_label> have the value of <Y_label> for <legend-label1> greater than the value of <Y_label> for <legend-label2>, arranged in increasing order of difference between the <Y_label> for <legend-label1> and <Y_label> for <legend-label2>?"
                            st = st.replace("<plural form of X_label>","'"+js['task7']['x_label']+"'")
                            st = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                            for i in range(len(legends_lab)-1):
                                for j in range(i+1,len(legends_lab)):
                                    leg1,leg2 = list(),list()
                                    for k1 in js['task6']['output']['data series']:
                                        if k1['name'].split() == legends_lab[i].split():
                                            for l1 in k1['data']:
                                                if 'y' in l1.keys():
                                                    leg1.append([l1['x'],l1['y']])
                                                elif 'y2' in l1.keys():
                                                    leg1.append([l1['x'],l1['y2']])
                                        elif k1['name'].split() == legends_lab[j].split():
                                            for l2 in k1['data']:
                                                if 'y' in l2.keys():
                                                    leg2.append([l2['x'],l2['y']])
                                                elif 'y2' in l2.keys():
                                                    leg2.append([l2['x'],l2['y2']])
                                    final_dict = dict()
                                    for a in range(len(leg1)):
                                        for b in range(len(leg2)):
                                            if leg1[a][0].split() == leg2[b][0].split():
                                                final_dict[leg1[a][0]] = [leg1[a][1],leg2[b][1]]
                                    dict_diff = dict()
                                    for a in final_dict:
                                        if final_dict[a][0]>final_dict[a][1]:
                                            dict_diff[a] = abs(final_dict[a][0]-final_dict[a][1])
                                    dict_diff = dict(sorted(dict_diff.items(),key = lambda x:x[0],reverse = True))
                                    qu = st.replace("<legend-label1>",legends_lab[i])
                                    if len(list(dict_diff.keys()))>0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '104a'
                                        q['question'] = qu.replace("<legend-label2>",legends_lab[j])
                                        q['answer'] = tuple(dict_diff.keys())
                                        qa.append(q)
                                        
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='bars':
                        legends_lab = js['task7']['legends']
                        if len(legends_lab)>1:
                            st = "For which <plural form of X_label> are the <Y_label> for <legend-label1> lesser than <Y_label> for <legend-label2>?"
                            st = st.replace("<plural form of X_label>","'"+js['task7']['x_label']+"'")
                            st = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                            for i in range(len(legends_lab)-1):
                                for j in range(i+1,len(legends_lab)):
                                    leg1,leg2 = list(),list()
                                    for k1 in js['task6']['output']['data series']:
                                        if k1['name'].split() == legends_lab[i].split():
                                            for l1 in k1['data']:
                                                if 'y' in l1.keys():
                                                    leg1.append([l1['x'],l1['y']])
                                                elif 'y2' in l1.keys():
                                                    leg1.append([l1['x'],l1['y2']])
                                        elif k1['name'].split() == legends_lab[j].split():
                                            for l2 in k1['data']:
                                                if 'y' in l2.keys():
                                                    leg2.append([l2['x'],l2['y']])
                                                elif 'y2' in l2.keys():
                                                    leg2.append([l2['x'],l2['y2']])
                                    final_dict = dict()
                                    for a in range(len(leg1)):
                                        for b in range(len(leg2)):
                                            if leg1[a][0].split() == leg2[b][0].split():
                                                final_dict[leg1[a][0]] = [leg1[a][1],leg2[b][1]]
                                    ans = list()
                                    for a in final_dict:
                                        if final_dict[a][0]<final_dict[a][1]:
                                            ans.append(a)
                                    qu = st.replace("<legend-label1>",legends_lab[i])
                                    if len(ans)>0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = str(105)
                                        q['question'] = qu.replace("<legend-label2>",legends_lab[j])
                                        q['answer'] = tuple(ans)
                                        qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'legends' in js['task7'].keys() and 'y_label' in js['task7'].keys() and 'x_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='bars':
                        legends_lab = js['task7']['legends']
                        if len(legends_lab)>1:
                            st = "Which <plural form of X_label> have the value of <Y_label> for <legend-label1> lesser than value of <Y_label> for <legend-label2> , arranged in increasing order of difference between the value of <Y_label> for <legend-label1> and than value of <Y_label> for <legend-label2>?"
                            st = st.replace("<plural form of X_label>","'"+js['task7']['x_label']+"'")
                            st = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                            for i in range(len(legends_lab)-1):
                                for j in range(i+1,len(legends_lab)):
                                    leg1,leg2 = list(),list()
                                    for k1 in js['task6']['output']['data series']:
                                        if k1['name'].split() == legends_lab[i].split():
                                            for l1 in k1['data']:
                                                if 'y' in l1.keys():
                                                    leg1.append([l1['x'],l1['y']])
                                                elif 'y2' in l1.keys():
                                                    leg1.append([l1['x'],l1['y2']])
                                        elif k1['name'].split() == legends_lab[j].split():
                                            for l2 in k1['data']:
                                                if 'y' in l2.keys():
                                                    leg2.append([l2['x'],l2['y']])
                                                elif 'y2' in l2.keys():
                                                    leg2.append([l2['x'],l2['y2']])
                                    final_dict = dict()
                                    for a in range(len(leg1)):
                                        for b in range(len(leg2)):
                                            if leg1[a][0].split() == leg2[b][0].split():
                                                final_dict[leg1[a][0]] = [leg1[a][1],leg2[b][1]]
                                    dict_diff = dict()
                                    for a in final_dict:
                                        if final_dict[a][0]<final_dict[a][1]:
                                            dict_diff[a] = abs(final_dict[a][0]-final_dict[a][1])
                                    dict_diff = dict(sorted(dict_diff.items(),key = lambda x:x[0],reverse = True))
                                    qu = st.replace("<legend-label1>",legends_lab[i])
                                    if len(list(dict_diff.keys()))>0:
                                        q=dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = '105a'
                                        q['question'] = qu.replace("<legend-label2>",legends_lab[j])
                                        q['answer'] = tuple(dict_diff.keys())
                                        qa.append(q)
                                        
            if 'task7' in js.keys() and 'task2' in js.keys():
                if 'x_label' in js['task7'].keys() and js['task2']!=None:
                    x_lab = js['task7']['x_label']
                    for i in js['task2']['output']['text_blocks']:
                        if i['text'] == x_lab:
                            w,h = abs(i['polygon']['x0'] - i['polygon']['x1']),abs(i['polygon']['y0'] - i['polygon']['y2'])
                    q=dict()
                    q['taxonomy id'] = '2a'
                    q['QID'] = 82
                    q['question'] = "What is the orientation of the text of independent axis title?"
                    q['answer'] = "Horizontal" if w>=h else "Vertical"
                    qa.append(q)
                    
            if 'task7' in js.keys() and 'task2' in js.keys():
                if 'y_label' in js['task7'].keys() and js['task2']!=None:
                    y_lab = js['task7']['y_label']
                    for i in js['task2']['output']['text_blocks']:
                        if i['text'] == y_lab:
                            w,h = abs(i['polygon']['x0'] - i['polygon']['x1']),abs(i['polygon']['y0'] - i['polygon']['y2'])
                    q=dict()
                    q['taxonomy id'] = '2c'
                    q['QID'] = 87
                    q['question'] = "What is the orientation of text of dependent axis title?"
                    q['answer'] = "Horizontal" if w>=h else "Vertical"
                    qa.append(q)
                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if 'figure_type' in js['task7'].keys() and js['task6']!=None and 'y_label' in js['task7'].keys():
                    if js['task7']['figure_type']=='lines':
                        legends_lab = list()
                        st = "Between <legend_label1> and <legend_label2>, which one has the peak value of <Y_label>?"
                        st = st.replace("<Y_label>","'"+js['task7']['y_label']+"'")
                        if 'legends' in js['task7'].keys():
                            legends_lab = js['task7']['legends']
                        elif 'mark_labels_lst' in js['task7'].keys():
                            legends_lab = js['task7']['mark_labels_lst']
                        if len(legends_lab)>1:
                            for i in range(len(legends_lab)-1):
                                for j in range(i+1,len(legends_lab)):
                                    lst_dat1,lst_dat2 = list(),list()
                                    for k in js['task6']['output']['data series']:
                                        if k['name']==legends_lab[i]:
                                            for k1 in k['data']:
                                                if 'y' in k1.keys():
                                                    lst_dat1.append(k1['y'])
                                                elif 'y2' in k1.keys():
                                                    lst_dat1.append(k1['y2'])
                                        elif k['name']==legends_lab[j]:
                                            for k1 in k['data']:
                                                if 'y' in k1.keys():
                                                    lst_dat2.append(k1['y'])
                                                elif 'y2' in k1.keys():
                                                    lst_dat2.append(k1['y2'])
                                    if len(lst_dat1)!=0 and len(lst_dat2)!=0:
                                        peak1 = max(lst_dat1)
                                        peak2 = max(lst_dat2)
                                        q = dict()
                                        q['taxonomy id'] = '4b'
                                        q['QID'] = 118
                                        qu = st.replace("<legend_label1>",legends_lab[i])
                                        q['question'] = qu.replace("<legend_label2>",legends_lab[j])
                                        if peak1>peak2:
                                            q['answer'] = legends_lab[i]
                                        else:
                                            q['answer'] = legends_lab[j]
                                        qa.append(q)
                                        
            if 'task7' in js.keys():
                if 'figure_type' in js['task7'].keys():
                    ticks = list()
                    if 'y_ticks_from_left' in js['task7'].keys():
                        ticks = js['task7']['y_ticks_from_left']
                    elif 'y_ticks_from_top' in js['task7'].keys():
                        ticks = js['task7']['y_ticks_from_top']
                    if len(ticks)!=0:
                        q=dict()
                        q['taxonomy id'] = '2c'
                        q['QID'] = 85
                        q['question'] = "Is dependent axis on a logarithmic scale or Linear scale?"
                        tick_lst = list()
                        for i in ticks:
                            try:
                                tick_lst.append(float(i))
                            except:
                                pass
                        if len(tick_lst)!=0:
                            diff = list()
                            for i in range(len(tick_lst)-1):
                                diff.append(abs(tick_lst[i+1]-tick_lst[i]))
                            q['answer'] = 'Linear' if all(x in [diff[0],diff[0]+1,diff[0]-1] for x in diff) else 'Logarithmic'
                            qa.append(q)
                            
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'x_label' in js['task7'].keys() and 'figure_type' in js['task7'].keys():
                    legends_lab = list()
                    st = "Which <X_label> has the maximum difference between <legend_label1> and <legend_label2>?"
                    st = st.replace("<X_label>","'"+js['task7']['x_label']+"'")
                    if 'legends' in js['task7'].keys():
                        legends_lab = js['task7']['legends']
                    if len(legends_lab)>1 and js['task7']['figure_type'] == 'bars':
                        for i in range(len(legends_lab)-1):
                            for j in range(i+1,len(legends_lab)):
                                q=dict()
                                q['taxonomy id'] = '4b'
                                q['QID'] = 92
                                qu = st.replace("<legend_label1>",legends_lab[i])
                                q['question'] = qu.replace("<legend_label2>",legends_lab[j])
                                lst1,lst2 = list(),list()
                                for k1 in js['task6']['output']['data series']:
                                    if k1['name'] == legends_lab[i]:
                                        for k2 in k1['data']:
                                            if 'y' in k2.keys():
                                                lst1.append([k2['x'],k2['y']])
                                            elif 'y2' in k2.keys():
                                                lst1.append([k2['x'],k2['y2']])
                                    elif k1['name'] == legends_lab[j]:
                                        for k2 in k1['data']:
                                            if 'y' in k2.keys():
                                                lst2.append([k2['x'],k2['y']])
                                            elif 'y2' in k2.keys():
                                                lst2.append([k2['x'],k2['y2']])
                                if len(lst1)==len(lst2) and len(lst1)!=0:
                                    diff = list()
                                    for k1 in range(len(lst1)):
                                        diff.append([lst1[k1][0],abs(lst1[k1][1] - lst2[k1][1])])
                                    diff = sorted(diff,key=lambda x:x[1],reverse = True)
                                    answers_list = list()
                                    for a in diff:
                                        if a[1] == diff[0][1]:
                                            answers_list.append(a[0])
                                    q['answer'] = ",".join(answers_list)
                                    qa.append(q)
                                    
            if 'task7' in js.keys() and 'task6' in js.keys():
                if js['task6']!=None and 'figure_type' in js['task7'].keys():
                    if js['task7']['figure_type'] == 'lines':
                        legends_lab = list()
                        st = "What value of major tick on independent axis has the maximum difference between <legend_label1> and <legend_label2>?"
                        if 'legends' in js['task7'].keys():
                            legends_lab = js['task7']['legends']
                        elif 'mark_labels_lst' in js['task7'].keys():
                            legends_lab = js['task7']['mark_labels_lst']
                        if len(legends_lab)>1 and 'extrapolated' in js['task7'].keys():
                            for i in range(len(legends_lab)-1):
                                for j in range(i+1,len(legends_lab)):
                                    q=dict()
                                    q['taxonomy id'] = '4b'
                                    q['QID'] = 93
                                    qu = st.replace("<legend_label1>",legends_lab[i])
                                    q['question'] = qu.replace("<legend_label2>",legends_lab[j])
                                    lst1,lst2 = list(),list()
                                    for k1 in js['task7']['extrapolated'].keys():
                                        if k1 == legends_lab[i]:
                                            for k2 in js['task7']['extrapolated'][k1]:
                                                lst1.append([k2[0],k2[1]])
                                        elif k1 == legends_lab[j]:
                                            for k2 in js['task7']['extrapolated'][k1]:
                                                lst2.append([k2[0],k2[1]])
                                    if len(lst1)==len(lst2) and len(lst1)!=0:
                                        diff = list()
                                        for k1 in range(len(lst1)):
                                            diff.append(abs(lst1[k1][1] - lst2[k1][1]))
                                        inde = diff.index(max(diff))
                                        q['answer'] = lst1[inde][0]
                                        qa.append(q)
                
        obj = json.dumps(qa,indent = 4)
        with open(qa_folder+file,'w') as f:
            f.write(obj)

C:\Users\pande\AppData\Local\Temp\ipykernel_20892\3935117908.py:4988: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  leg_lab_values[i] = stats_mode.mode(dat)
C:\Users\pande\AppData\Local\Temp\ipykernel_20892\3935117908.py:5016: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this wa

In [13]:
for i in lst_of_folders:
    qa_folder = "C:/Users/pande/Desktop/test_2023/QA/"+i+"_qa/"
    for j in os.listdir(qa_folder):
        with open(qa_folder+j,'r') as f:
            js = json.load(f)
            for k in js:
                k['QID'] = str(k['QID'])
                k['question'] = k['question'].replace("\n"," ")
                if type(k['answer']) == str:
                    k['answer'] = k['answer'].replace("\n"," ")
        os.remove(qa_folder+j)
        with open(qa_folder+j, 'w') as f:
            json.dump(js, f, indent=4)

In [14]:
import re
for i in lst_of_folders:
    qa_folder = "C:/Users/pande/Desktop/test_2023/QA/"      # path for QA folder
    for j in os.listdir(qa_folder+i+"_qa/"):
        with open(qa_folder+i+"_qa/"+j) as f:
            js = json.load(f)
            for k in js:
                k['question'] = re.sub(r"[\n]", " ", k['question'])
                if type(k['answer']) == str:
                    k['answer'] = re.sub(r"[\n]", " ", k['answer'])
                elif type(k['answer']) == list:
                    lst = list()
                    for a in k['answer']:
                        if type(a) == str:
                            a = re.sub(r"[\n]", " ", a)
                            lst.append(a)
                            k['answer'] = lst
        os.remove(qa_folder+i+"_qa/"+j)
        with open(qa_folder+i+"_qa/"+j, 'w') as f:
            json.dump(js, f, indent=4)

In [18]:
for i in lst_of_folders:
    qa_folder = "C:/Users/pande/Desktop/test_2023/QA/"+i+"_qa/"
    for j in os.listdir(qa_folder):
        with open(qa_folder+j,'r') as f:
            js = json.load(f)
            for k in js:
                if k['answer'] == "":
                    print(j)
                elif k['answer'] is None:
                    print(j)
                if 'answer_type' not in k.keys():
                    print(j)

In [16]:
df = pd.read_csv("C:/Users/pande/Desktop/template.csv")
df = pd.DataFrame(df)
for k in lst_of_folders:
    qa_folder = "C:/Users/pande/Desktop/test_2023/QA/"      # path for QA folder
    for j in os.listdir(qa_folder+k+"_qa/"):
        with open(qa_folder+k+"_qa/"+j) as f:
            js = json.load(f)
            for k1 in js:
                k1['answer_type'] = list(df.iloc[np.where(df['qid']==k1['QID'])]['answer_type'])[0]
        os.remove(qa_folder+k+"_qa/"+j)
        with open(qa_folder+k+"_qa/"+j, 'w') as f:
            json.dump(js, f, indent=4)